In [9]:
import numpy as np
import subprocess
import os
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
from datetime import datetime

# =============================================================================
# CONFIGURATION
# =============================================================================

# XFOIL settings
XFOIL_EXE = "Downloads/XFOIL6.99/xfoil.exe" # Change to "xfoil" on Mac/Linux
REYNOLDS_NUMBER = 500000
ANGLE_OF_ATTACK = 4.0
MAX_ITER = 100  # XFOIL iterations

# Optimization settings
OPTIMIZATION_ITERATIONS = 100  # Start with 100, increase to 200 later
POPULATION_SIZE = 15
BOUNDS = [
    (0.00, 0.09),  # m
    (0.30, 0.70),  # p  (narrowed)
    (0.08, 0.18),  # t  (narrowed)
]

# Output directory
OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =============================================================================
# NACA 4-DIGIT AIRFOIL GENERATOR
# =============================================================================

def generate_naca_4digit(m, p, t, n_points=200):

    # Cosine spacing for better resolution near leading edge
    beta = np.linspace(0, np.pi, n_points)
    x = 0.5 * (1 - np.cos(beta))
    
    # Thickness distribution (symmetric)
    yt = 5 * t * (0.2969 * np.sqrt(x) - 0.1260 * x - 0.3516 * x**2 + 
                   0.2843 * x**3 - 0.1015 * x**4)
    
    # Mean camber line
    yc = np.zeros_like(x)
    dyc_dx = np.zeros_like(x)
    
    if m > 0 and p > 0:  # Cambered airfoil
        # Forward of maximum camber
        mask1 = x < p
        yc[mask1] = m / p**2 * (2 * p * x[mask1] - x[mask1]**2)
        dyc_dx[mask1] = 2 * m / p**2 * (p - x[mask1])
        
        # Aft of maximum camber
        mask2 = x >= p
        yc[mask2] = m / (1 - p)**2 * ((1 - 2 * p) + 2 * p * x[mask2] - x[mask2]**2)
        dyc_dx[mask2] = 2 * m / (1 - p)**2 * (p - x[mask2])
    
    # Angle of camber line
    theta = np.arctan(dyc_dx)
    
    # Upper and lower surface coordinates
    x_upper = x - yt * np.sin(theta)
    y_upper = yc + yt * np.cos(theta)
    
    x_lower = x + yt * np.sin(theta)
    y_lower = yc - yt * np.cos(theta)
    
    return x_upper, y_upper, x_lower, y_lower

def write_airfoil_file(filename, x_upper, y_upper, x_lower, y_lower):
    """Write airfoil coordinates to XFOIL .dat with TE->LE (upper), LE->TE (lower)."""
    with open(filename, 'w') as f:
        f.write("NACA4\n")
        # Upper surface: TE -> LE
        for i in range(len(x_upper)-1, -1, -1):
            f.write(f"{x_upper[i]:.6f} {y_upper[i]:.6f}\n")
        # Lower surface: LE -> TE
        for i in range(len(x_lower)):
            f.write(f"{x_lower[i]:.6f} {y_lower[i]:.6f}\n")

# =============================================================================
# XFOIL INTERFACE
# =============================================================================

import tempfile
import shutil

def _parse_polar_file(polar_path):
    data = []
    if not os.path.exists(polar_path):
        return data
    with open(polar_path) as f:
        for line in f:
            line=line.strip()
            if not line or line.startswith('#') or line.startswith('x'):
                continue
            sp = line.split()
            # Expect: alpha  CL    CD     CDp    CM    Top_Xtr Bot_Xtr
            try:
                a = float(sp[0]); CL = float(sp[1]); CD = float(sp[2]); CM = float(sp[4])
                data.append((a, CL, CD, CM))
            except:
                pass
    return data

def run_xfoil(airfoil_file, reynolds, aseq=None, alpha=None,
              ncrit=NCRIT, mach=MACH, max_iter=MAX_ITER, timeout=TIMEOUT):
    """
    Robust XFOIL call for Windows builds:
      - work in a temp dir
      - CRLF line endings
      - use 'PACC out.pol' to avoid filename prompt
      - close with bare 'PACC' at end
      - leave OPER before QUIT
      - fallback to stdout parsing if no polar
    Returns list[(alpha, CL, CD, CM)].
    """
    tmpdir = tempfile.mkdtemp(prefix="xf_")
    keep_dir = DEBUG_XFOIL
    try:
        local_dat = os.path.join(tmpdir, "foil.dat")
        shutil.copyfile(airfoil_file, local_dat)
        polar_path = os.path.join(tmpdir, "out.pol")

        def build_script(iter_cap, use_seq=True):
            lines = [
                f"LOAD foil.dat",
                "PANE",
                "OPER",
                f"VISC {int(reynolds)}",
                f"VPAR N {int(ncrit)}",
                f"MACH {mach}",
                f"ITER {int(iter_cap)}",
                # one-line polar open (prevents interactive filename prompt)
                f"PACC {os.path.basename(polar_path)}",
            ]
            if use_seq and aseq is not None:
                a0, a1, da = aseq
                lines.append(f"ASEQ {a0} {a1} {da}")
            elif alpha is not None:
                lines.append(f"ALFA {alpha}")
            else:
                lines.append("ALFA 0.0")
            # close polar file and leave OPER before QUIT
            lines += [
                "PACC",   # close polar accumulation
                "",       # blank line returns from OPER submenu in many builds
                "QUIT",
            ]
            # IMPORTANT: CRLF endings for some Windows builds
            return "\r\n".join(lines) + "\r\n"

        attempts = [(max_iter, True), (max_iter*2, True), (max_iter*2, False)]
        last_stdout = ""
        for iters, use_seq in attempts:
            script = build_script(iters, use_seq)
            cp = subprocess.run(
                [XFOIL_EXE],
                input=script,
                text=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=timeout,
                check=False,
                cwd=tmpdir,
            )
            last_stdout = cp.stdout
            if DEBUG_XFOIL:
                open(os.path.join(tmpdir, "xfoil_script.inp"), "w", newline="").write(script)
                open(os.path.join(tmpdir, "stdout.txt"), "w", newline="").write(cp.stdout)
                open(os.path.join(tmpdir, "stderr.txt"), "w", newline="").write(cp.stderr)
                print(f"[DEBUG] Temp dir: {tmpdir}")
                print("[DEBUG] XFOIL stdout tail:\n" + "\n".join(cp.stdout.splitlines()[-12:]))

            # try polar first
            data = _parse_polar_file(polar_path)
            if data:
                return data

            # fallback: parse stdout single-point lines
            sp = _parse_stdout_for_singlepoint(cp.stdout)
            if sp:
                return sp

        if DEBUG_XFOIL:
            print("[DEBUG] No polar and no parseable stdout. Last stdout tail:\n" +
                  "\n".join(last_stdout.splitlines()[-20:]))
        return []
    finally:
        if not keep_dir:
            shutil.rmtree(tmpdir, ignore_errors=True)

# =============================================================================
# OPTIMIZATION
# =============================================================================

# Global variables to track optimization progress
eval_count = 0
best_fitness = -np.inf
fitness_history = []
design_history = []

from functools import lru_cache

# tighten bounds in main too (see section 5)
ALPHA_BAND = (-2.0, 8.0, 1.0)  # a0, a1, da

BAD_FIT = 1e3  # finite penalty

@lru_cache(maxsize=5000)
def evaluate_tuple(m, p, t):
    # Build airfoil
    x_u, y_u, x_l, y_l = generate_naca_4digit(m, p, t)
    tmp_path = os.path.join(OUTPUT_DIR, f"foil_{hash((round(m,4),round(p,4),round(t,4)))}.dat")
    write_airfoil_file(tmp_path, x_u, y_u, x_l, y_l)
    pol = run_xfoil(tmp_path, REYNOLDS_NUMBER, aseq=ALPHA_BAND, max_iter=MAX_ITER)
    if not pol:  # fail
        return None

    # Compute max L/D with guard
    best = None
    for a, CL, CD, CM in pol:
        if CD > 0:
            ld = CL / CD
            if best is None or ld > best[0]:
                best = (ld, a, CL, CD, CM)
    return best  # (ld, a, CL, CD, CM) or None

def objective_function(x):
    global eval_count, best_fitness
    eval_count += 1
    m, p, t = x

    # quick feasibility screen to reduce XFOIL crashes
    if not (0.00 <= m <= 0.09 and 0.30 <= p <= 0.70 and 0.08 <= t <= 0.18):
        fitness_history.append(0)
        design_history.append([m, p, t, 0, 0, 0])
        return BAD_FIT

    res = evaluate_tuple(float(m), float(p), float(t))
    if res is None:
        print(f"Eval {eval_count}: XFOIL failed for m={m:.3f}, p={p:.3f}, t={t:.3f}")
        fitness_history.append(0)
        design_history.append([m, p, t, 0, 0, 0])
        return BAD_FIT
    ld, a, CL, CD, CM = res
    best_fitness = max(best_fitness, ld)
    fitness_history.append(ld)
    design_history.append([m, p, t, CL, CD, ld])

    naca_designation = f"{int(m*100):02d}{int(p*10):1d}{int(t*100):02d}"
    print(f"Eval {eval_count}: NACA {naca_designation:>4s} | α*={a:4.1f} | "
          f"CL={CL:6.3f} | CD={CD:7.4f} | L/D={ld:7.2f} | Best={best_fitness:7.2f}")

    return -ld  # minimize


def plot_results(result, design_history):
    """Generate all result plots."""
    
    # Convert history to array
    history = np.array(design_history)
    
    # Create figure with subplots
    fig = plt.figure(figsize=(15, 10))
    
    # 1. Convergence history
    ax1 = plt.subplot(2, 3, 1)
    iterations = range(1, len(fitness_history) + 1)
    ax1.plot(iterations, fitness_history, 'b-', alpha=0.3, label='All evaluations')
    
    # Running best
    running_best = [max(fitness_history[:i+1]) for i in range(len(fitness_history))]
    ax1.plot(iterations, running_best, 'r-', linewidth=2, label='Best so far')
    
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('L/D Ratio')
    ax1.set_title('Optimization Convergence')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Best airfoil shape
    ax2 = plt.subplot(2, 3, 2)
    best_m, best_p, best_t = result.x
    x_u, y_u, x_l, y_l = generate_naca_4digit(best_m, best_p, best_t)
    
    ax2.plot(x_u, y_u, 'b-', linewidth=2, label='Upper surface')
    ax2.plot(x_l, y_l, 'r-', linewidth=2, label='Lower surface')
    ax2.set_xlabel('x/c')
    ax2.set_ylabel('y/c')
    ax2.set_title(f'Best Airfoil: NACA {int(best_m*100)}{int(best_p*10)}{int(best_t*100)}')
    ax2.axis('equal')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # 3. Cl vs Cd scatter
    ax3 = plt.subplot(2, 3, 3)
    valid_mask = history[:, 3] > 0  # Where Cl > 0 (valid runs)
    ax3.scatter(history[valid_mask, 4], history[valid_mask, 3], 
                c=history[valid_mask, 5], cmap='viridis', alpha=0.6)
    ax3.set_xlabel('Cd (Drag Coefficient)')
    ax3.set_ylabel('Cl (Lift Coefficient)')
    ax3.set_title('Cl vs Cd (colored by L/D)')
    ax3.grid(True, alpha=0.3)
    cbar = plt.colorbar(ax3.collections[0], ax=ax3)
    cbar.set_label('L/D Ratio')
    ax4 = plt.subplot(2, 3, 4)
    scatter = ax4.scatter(history[valid_mask, 0]*100, history[valid_mask, 2]*100,
                         c=history[valid_mask, 5], cmap='viridis', alpha=0.6)
    ax4.scatter(best_m*100, best_t*100, color='red', s=200, marker='*', 
                edgecolors='black', linewidths=2, label='Best', zorder=5)
    ax4.set_xlabel('Maximum Camber (%)')
    ax4.set_ylabel('Maximum Thickness (%)')
    ax4.set_title('Design Space Exploration')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Parameter distributions
    ax5 = plt.subplot(2, 3, 5)
    ax5.hist(history[valid_mask, 5], bins=20, alpha=0.7, edgecolor='black')
    ax5.axvline(best_fitness, color='red', linestyle='--', linewidth=2, label='Best')
    ax5.set_xlabel('L/D Ratio')
    ax5.set_ylabel('Frequency')
    ax5.set_title('L/D Distribution')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. Comparison with initial design
    ax6 = plt.subplot(2, 3, 6)
    # Use NACA 2412 as baseline
    x_u_base, y_u_base, x_l_base, y_l_base = generate_naca_4digit(0.02, 0.4, 0.12)
    
    ax6.plot(x_u_base, y_u_base, 'gray', linewidth=1.5, label='NACA 2412 (baseline)', alpha=0.7)
    ax6.plot(x_l_base, y_l_base, 'gray', linewidth=1.5, alpha=0.7)
    ax6.plot(x_u, y_u, 'b-', linewidth=2, label='Optimized')
    ax6.plot(x_l, y_l, 'b-', linewidth=2)
    ax6.set_xlabel('x/c')
    ax6.set_ylabel('y/c')
    ax6.set_title('Baseline vs Optimized')
    ax6.axis('equal')
    ax6.grid(True, alpha=0.3)
    ax6.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'optimization_results.png'), dpi=300, bbox_inches='tight')
    print(f"\nPlots saved to {OUTPUT_DIR}/optimization_results.png")
    plt.show()

def main():
    """Main optimization routine."""
    
    print("="*70)
    print("AIRFOIL OPTIMIZATION WITH XFOIL")
    print("="*70)
    print(f"Reynolds Number: {REYNOLDS_NUMBER}")
    print(f"Angle of Attack: {ANGLE_OF_ATTACK}°")
    print(f"Optimization Iterations: {OPTIMIZATION_ITERATIONS}")
    print(f"Population Size: {POPULATION_SIZE}")
    print(f"Design Variables: M (camber), P (position), T (thickness)")
    print(f"Objective: Maximize L/D ratio")
    print("="*70)
    print()
    
    # Test XFOIL first
    print("Testing XFOIL with NACA 2412...")
    x_u, y_u, x_l, y_l = generate_naca_4digit(0.02, 0.4, 0.12)
    test_file = os.path.join(OUTPUT_DIR, "test_airfoil.dat")
    write_airfoil_file(test_file, x_u, y_u, x_l, y_l)
    cl_test, cd_test = run_xfoil(test_file, REYNOLDS_NUMBER, ANGLE_OF_ATTACK, MAX_ITER)
    
    if cl_test is None:
        print("\n❌ ERROR: XFOIL test failed!")
        print("Make sure xfoil.exe is in the same folder or in your PATH")
        return
    else:
        print(f"✅ XFOIL working! NACA 2412: Cl={cl_test:.3f}, Cd={cd_test:.5f}, L/D={cl_test/cd_test:.2f}")
        print()
    
    # Run optimization
    print("Starting optimization...")
    print("-"*70)
    
    start_time = datetime.now()
    
    result = differential_evolution(
    objective_function,
    BOUNDS,
    maxiter=OPTIMIZATION_ITERATIONS,
    popsize=POPULATION_SIZE,
    strategy='best1bin',
    seed=42,
    disp=False,
    workers=-1   # parallel
)

    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    # Extract best design
    best_m, best_p, best_t = result.x
    best_naca = f"{int(best_m*100)}{int(best_p*10)}{int(best_t*100)}"
    
    # Get best performance
    history = np.array(design_history)
    best_idx = np.argmax(history[:, 5])
    best_cl = history[best_idx, 3]
    best_cd = history[best_idx, 4]
    best_ld = history[best_idx, 5]
    
    # Print results
    print()
    print("="*70)
    print("OPTIMIZATION COMPLETE!")
    print("="*70)
    print(f"Time elapsed: {duration:.1f} seconds ({duration/60:.1f} minutes)")
    print(f"Total evaluations: {eval_count}")
    print(f"Successful evaluations: {np.sum(history[:, 5] > 0)}")
    print()
    print(f"Best Airfoil: NACA {best_naca}")
    print(f"  M (camber) = {best_m:.4f} ({best_m*100:.2f}%)")
    print(f"  P (position) = {best_p:.4f} ({best_p*100:.1f}%)")
    print(f"  T (thickness) = {best_t:.4f} ({best_t*100:.2f}%)")
    print()
    print(f"Performance:")
    print(f"  Cl (lift)  = {best_cl:.4f}")
    print(f"  Cd (drag)  = {best_cd:.6f}")
    print(f"  L/D ratio  = {best_ld:.2f}")
    print()
    print(f"Improvement over NACA 2412: {((best_ld/(cl_test/cd_test))-1)*100:+.1f}%")
    print("="*70)
    
    # Save results to file
    results_file = os.path.join(OUTPUT_DIR, "optimization_results.txt")
    with open(results_file, 'w') as f:
        f.write("AIRFOIL OPTIMIZATION RESULTS\n")
        f.write("="*70 + "\n\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Reynolds Number: {REYNOLDS_NUMBER}\n")
        f.write(f"Angle of Attack: {ANGLE_OF_ATTACK}°\n\n")
        f.write(f"Best Airfoil: NACA {best_naca}\n")
        f.write(f"  M = {best_m:.4f}\n")
        f.write(f"  P = {best_p:.4f}\n")
        f.write(f"  T = {best_t:.4f}\n\n")
        f.write(f"Performance:\n")
        f.write(f"  Cl = {best_cl:.4f}\n")
        f.write(f"  Cd = {best_cd:.6f}\n")
        f.write(f"  L/D = {best_ld:.2f}\n\n")
        f.write(f"Evaluations: {eval_count}\n")
        f.write(f"Time: {duration:.1f} seconds\n")
    
    print(f"\nResults saved to {results_file}")
    
    # Generate plots
    print("\nGenerating plots...")
    plot_results(result, design_history)
    
    print("\n✅ All done! Check the 'results' folder for outputs.")

if __name__ == "__main__":
    main()

AIRFOIL OPTIMIZATION WITH XFOIL
Reynolds Number: 500000
Angle of Attack: 4.0°
Optimization Iterations: 100
Population Size: 15
Design Variables: M (camber), P (position), T (thickness)
Objective: Maximize L/D ratio

Testing XFOIL with NACA 2412...


TypeError: cannot unpack non-iterable float object

In [13]:
import os
import math
import shutil
import tempfile
import subprocess
from datetime import datetime
from functools import lru_cache

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution

# =============================================================================
# CONFIGURATION
# =============================================================================

# --- XFOIL executable (adjust if needed)
# Windows example: r"C:\Users\you\Desktop\xfoil\xfoil.exe"
XFOIL_EXE = "Downloads/XFOIL6.99/xfoil.exe"     # use "xfoil" on Mac/Linux if in PATH

# Operating point
REYNOLDS_NUMBER = 500_000
NCRIT = 9
MACH = 0.0
MAX_ITER = 200             # XFOIL iterations cap
TIMEOUT = 60               # seconds for each XFOIL run

# AoA sweep for robustness (a0, a1, da)
ALPHA_BAND = (-2.0, 8.0, 1.0)

# Optimization
OPTIMIZATION_ITERATIONS = 120     # DE generations
POPULATION_SIZE = 15              # popsize multiplier (SciPy meaning: individuals = popsize * dims)
F_MUT = 0.7
CR = 0.9
RANDOM_SEED = 42

# Design variable bounds (tighter = fewer XFOIL fails)
# m: max camber, p: location, t: thickness (all fractions of chord)
BOUNDS = [
    (0.00, 0.09),   # m
    (0.30, 0.70),   # p
    (0.08, 0.18),   # t
]

# Output
OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Penalty for failed evaluations (finite to keep DE stable)
BAD_FIT = 1e3

# =============================================================================
# GEOMETRY: NACA 4-DIGIT
# =============================================================================

def generate_naca_4digit(m, p, t, n_points=200):
    """
    Return airfoil coordinates as four arrays: x_upper, y_upper, x_lower, y_lower.
    Uses cosine spacing for x.
    """
    beta = np.linspace(0.0, np.pi, n_points)
    x = 0.5 * (1 - np.cos(beta))  # cosine spacing in [0,1]

    # Symmetric thickness distribution
    yt = 5.0 * t * (
        0.2969 * np.sqrt(np.clip(x, 1e-12, None))  # avoid sqrt(0) warnings
        - 0.1260 * x
        - 0.3516 * x**2
        + 0.2843 * x**3
        - 0.1015 * x**4
    )

    # Mean camber line yc and slope dyc/dx
    yc = np.zeros_like(x)
    dyc = np.zeros_like(x)

    if m > 0 and 0 < p < 1:
        mask1 = x < p
        mask2 = ~mask1
        # 0 <= x < p
        if p > 0:
            yc[mask1] = m / p**2 * (2 * p * x[mask1] - x[mask1] ** 2)
            dyc[mask1] = 2 * m / p**2 * (p - x[mask1])
        # p <= x <= 1
        if (1 - p) > 0:
            yc[mask2] = m / (1 - p) ** 2 * ((1 - 2 * p) + 2 * p * x[mask2] - x[mask2] ** 2)
            dyc[mask2] = 2 * m / (1 - p) ** 2 * (p - x[mask2])

    theta = np.arctan(dyc)

    # Upper and lower surfaces (offset normal to camber line)
    x_u = x - yt * np.sin(theta)
    y_u = yc + yt * np.cos(theta)

    x_l = x + yt * np.sin(theta)
    y_l = yc - yt * np.cos(theta)

    return x_u, y_u, x_l, y_l


def write_airfoil_file(filename, x_upper, y_upper, x_lower, y_lower):
    """
    Write .dat for XFOIL:
      - Upper surface: TE -> LE
      - Lower surface: LE -> TE
    This ordering helps XFOIL paneling.
    """
    with open(filename, "w") as f:
        f.write("NACA4\n")
        # Upper TE -> LE
        for i in range(len(x_upper) - 1, -1, -1):
            f.write(f"{x_upper[i]:.6f} {y_upper[i]:.6f}\n")
        # Lower LE -> TE
        for i in range(len(x_lower)):
            f.write(f"{x_lower[i]:.6f} {y_lower[i]:.6f}\n")

# =============================================================================
# XFOIL INTERFACE (POLAR-BASED)
# =============================================================================

import re

# line patterns (accepts scientific notation, varying spaces/case)
_RE_A_CL = re.compile(
    r"a\s*=\s*([+\-]?\d+(?:\.\d*)?)\s+.*?CL\s*=\s*([+\-]?\d+(?:\.\d*)?(?:[Ee][+\-]?\d+)?)",
    re.IGNORECASE,
)
_RE_CD = re.compile(
    r"\bCD\s*=\s*([+\-]?\d+(?:\.\d*)?(?:[Ee][+\-]?\d+)?)",
    re.IGNORECASE,
)
_RE_CM = re.compile(
    r"\bC[Mm]\s*=\s*([+\-]?\d+(?:\.\d*)?(?:[Ee][+\-]?\d+)?)",
    re.IGNORECASE,
)

def _parse_stdout_for_singlepoint(stdout_text):
    """
    Parse XFOIL console output where 'a & CL' appear on one line and 'Cm & CD'
    may appear on the NEXT line (as in your screenshot).
    Returns a list with one or more tuples: (alpha, CL, CD, CM).
    """
    rows = []
    cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}

    for line in stdout_text.splitlines():
        # First, look for 'a' and 'CL' on the same line
        m1 = _RE_A_CL.search(line)
        if m1:
            # whenever we see a new a/CL, flush the previous (if complete)
            if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
                rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
                cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}

            cur["a"]  = float(m1.group(1))
            cur["CL"] = float(m1.group(2))
            # continue to next line to find CD/CM
            continue

        # Then, on the same or next line, pick up CM and/or CD (any order)
        m_cd = _RE_CD.search(line)
        if m_cd:
            try:
                cur["CD"] = float(m_cd.group(1))
            except:
                pass

        m_cm = _RE_CM.search(line)
        if m_cm:
            try:
                cur["CM"] = float(m_cm.group(1))
            except:
                pass

        # If we've assembled a full record, store it
        if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
            rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
            cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}

    # In case the last pair wasn’t flushed but is complete
    if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
        rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))

    # Return the last (usually converged) result; you can return all if you prefer
    return rows[-1:] if rows else []


def run_xfoil(airfoil_file, reynolds, aseq=None, alpha=None,
              ncrit=NCRIT, mach=MACH, max_iter=MAX_ITER, timeout=TIMEOUT):
    """
    Call XFOIL and return a list of (alpha, CL, CD, CM).
    Use 'aseq=(a0,a1,da)' for a sweep (recommended), or 'alpha' for single point.
    """
    tmpdir = tempfile.mkdtemp(prefix="xf_")
    try:
        local_dat = os.path.join(tmpdir, os.path.basename(airfoil_file))
        shutil.copyfile(airfoil_file, local_dat)
        polar_path = os.path.join(tmpdir, "out.pol")

        def build_script(iter_cap):
            lines = [
                f"LOAD {local_dat}",
                "PANE",
                "OPER",
                f"VISC {reynolds:.0f}",
                f"VPAR N {int(ncrit)}",
                f"MACH {mach}",
                f"ITER {int(iter_cap)}",
                "PACC",
                polar_path,
                "",
            ]
            if aseq is not None:
                a0, a1, da = aseq
                lines.append(f"ASEQ {a0} {a1} {da}")
            elif alpha is not None:
                lines.append(f"ALFA {alpha}")
            else:
                lines.append("ALFA 0.0")
            lines += ["PACC", "", "", "QUIT"]
            return "\n".join(lines)

        # Try with base iter cap, then a retry with doubled iter cap
        for iter_cap in (max_iter, max_iter * 2):
            script = build_script(iter_cap)
            subprocess.run(
                [XFOIL_EXE],
                input=script,
                text=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=timeout,
                check=False,
            )
            data = _parse_polar_file(polar_path)
            if data:
                return data

        return []  # failure
    except Exception as e:
        print("XFOIL error:", e)
        return []
    finally:
        shutil.rmtree(tmpdir, ignore_errors=True)

# =============================================================================
# OBJECTIVE FUNCTION (robust max L/D across AoA band)
# =============================================================================

eval_count = 0
best_fitness = -np.inf          # tracks best L/D seen
fitness_history = []            # list of L/D per eval
design_history = []             # rows: [m, p, t, CL, CD, L/D]

@lru_cache(maxsize=4096)
def evaluate_tuple(m, p, t):
    """Run XFOIL once for (m,p,t) and return (best_ld, a*, CL, CD) or None."""
    x_u, y_u, x_l, y_l = generate_naca_4digit(m, p, t)
    dat_path = os.path.join(OUTPUT_DIR, f"foil_{hash((round(m,4),round(p,4),round(t,4)))}.dat")
    write_airfoil_file(dat_path, x_u, y_u, x_l, y_l)

    pol = run_xfoil(dat_path, REYNOLDS_NUMBER, aseq=ALPHA_BAND)
    if not pol:
        return None

    valid = [(CL / CD, a, CL, CD) for (a, CL, CD, CM) in pol if CD > 0]
    if not valid:
        return None

    return max(valid, key=lambda z: z[0])  # (ld, a*, CL, CD)


def objective_function(x):
    global eval_count, best_fitness
    eval_count += 1
    m, p, t = map(float, x)

    # Feasibility screen (helps XFOIL)
    if not (0.00 <= m <= 0.09 and 0.30 <= p <= 0.70 and 0.08 <= t <= 0.18):
        fitness_history.append(0.0)
        design_history.append([m, p, t, 0.0, 0.0, 0.0])
        print(f"Eval {eval_count}: bounds reject m={m:.3f}, p={p:.3f}, t={t:.3f}")
        return BAD_FIT

    res = evaluate_tuple(round(m,4), round(p,4), round(t,4))
    if res is None:
        fitness_history.append(0.0)
        design_history.append([m, p, t, 0.0, 0.0, 0.0])
        print(f"Eval {eval_count}: XFOIL failed for m={m:.3f}, p={p:.3f}, t={t:.3f}")
        return BAD_FIT

    ld, a_star, CL, CD = res
    best_fitness = max(best_fitness, ld)
    fitness_history.append(ld)
    design_history.append([m, p, t, CL, CD, ld])

    tag = f"{int(m*100):02d}{int(p*10):1d}{int(t*100):02d}"
    print(f"Eval {eval_count}: NACA {tag:>4s} | α*={a_star:4.1f} | "
          f"CL={CL:6.3f} | CD={CD:7.4f} | L/D={ld:7.2f} | Best={best_fitness:7.2f}")

    return -ld  # SciPy minimizes

# =============================================================================
# VISUALIZATION
# =============================================================================

def plot_results(result, design_history, test_ld):
    history = np.array(design_history)
    valid_mask = history[:, 5] > 0

    # Figure
    fig = plt.figure(figsize=(15, 10))

    # 1) Convergence
    ax1 = plt.subplot(2, 3, 1)
    iters = np.arange(1, len(fitness_history) + 1)
    ax1.plot(iters, fitness_history, alpha=0.35, label="All evals")
    running_best = np.maximum.accumulate(fitness_history)
    ax1.plot(iters, running_best, linewidth=2, label="Best so far")
    ax1.set_xlabel("Evaluation")
    ax1.set_ylabel("L/D")
    ax1.set_title("Optimization Convergence")
    ax1.grid(alpha=0.3)
    ax1.legend()

    # 2) Best airfoil geometry
    ax2 = plt.subplot(2, 3, 2)
    best_m, best_p, best_t = result.x
    x_u, y_u, x_l, y_l = generate_naca_4digit(best_m, best_p, best_t)
    ax2.plot(x_u, y_u, linewidth=2, label="Upper")
    ax2.plot(x_l, y_l, linewidth=2, label="Lower")
    ax2.set_aspect("equal", "box")
    ax2.grid(alpha=0.3)
    ax2.set_title(f"Best Airfoil: NACA {int(best_m*100):02d}{int(best_p*10):1d}{int(best_t*100):02d}")
    ax2.set_xlabel("x/c"); ax2.set_ylabel("y/c")
    ax2.legend()

    # 3) CL vs CD colored by L/D
    ax3 = plt.subplot(2, 3, 3)
    sc = ax3.scatter(history[valid_mask, 4], history[valid_mask, 3],
                     c=history[valid_mask, 5], cmap="viridis", alpha=0.7)
    ax3.set_xlabel("CD"); ax3.set_ylabel("CL")
    ax3.set_title("CL vs CD (color = L/D)")
    ax3.grid(alpha=0.3)
    cbar = plt.colorbar(sc, ax=ax3); cbar.set_label("L/D")

    # 4) Design space: m vs t
    ax4 = plt.subplot(2, 3, 4)
    sc2 = ax4.scatter(history[valid_mask, 0]*100, history[valid_mask, 2]*100,
                      c=history[valid_mask, 5], cmap="viridis", alpha=0.7)
    ax4.scatter(best_m*100, best_t*100, s=200, marker="*", edgecolors="k", label="Best", zorder=5)
    ax4.set_xlabel("Max camber m (%)"); ax4.set_ylabel("Max thickness t (%)")
    ax4.set_title("Design Space Exploration")
    ax4.grid(alpha=0.3); ax4.legend()

    # 5) L/D distribution
    ax5 = plt.subplot(2, 3, 5)
    ax5.hist(history[valid_mask, 5], bins=20, edgecolor="black", alpha=0.8)
    ax5.axvline(np.max(history[:,5]), color="r", linestyle="--", label="Best")
    ax5.axvline(test_ld, color="gray", linestyle=":", label="Baseline (2412)")
    ax5.set_xlabel("L/D"); ax5.set_ylabel("Frequency")
    ax5.set_title("L/D Distribution")
    ax5.grid(alpha=0.3); ax5.legend()

    # 6) Baseline vs optimized
    ax6 = plt.subplot(2, 3, 6)
    x_u_b, y_u_b, x_l_b, y_l_b = generate_naca_4digit(0.02, 0.4, 0.12)  # NACA 2412
    ax6.plot(x_u_b, y_u_b, alpha=0.8, label="NACA 2412 baseline")
    ax6.plot(x_l_b, y_l_b, alpha=0.8)
    ax6.plot(x_u, y_u, linewidth=2, label="Optimized")
    ax6.plot(x_l, y_l, linewidth=2)
    ax6.set_aspect("equal", "box")
    ax6.grid(alpha=0.3)
    ax6.set_xlabel("x/c"); ax6.set_ylabel("y/c")
    ax6.set_title("Baseline vs Optimized")
    ax6.legend()

    plt.tight_layout()
    out_png = os.path.join(OUTPUT_DIR, "optimization_results.png")
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    print(f"\nPlots saved to {out_png}")
    plt.show()

# =============================================================================
# MAIN
# =============================================================================

def main():
    print("=" * 70)
    print("AIRFOIL OPTIMIZATION WITH XFOIL (polar-based, AoA sweep)")
    print("=" * 70)
    print(f"Reynolds Number: {REYNOLDS_NUMBER}")
    print(f"Ncrit: {NCRIT} | Mach: {MACH}")
    print(f"AoA band: {ALPHA_BAND[0]}..{ALPHA_BAND[1]} step {ALPHA_BAND[2]} deg")
    print(f"Optimization: generations={OPTIMIZATION_ITERATIONS}, popsize={POPULATION_SIZE}")
    print(f"Bounds: m{BOUNDS[0]}, p{BOUNDS[1]}, t{BOUNDS[2]}")
    print("=" * 70, "\n")

    # --- Smoke test on NACA 2412
    print("Testing XFOIL with NACA 2412...")
    x_u, y_u, x_l, y_l = generate_naca_4digit(0.02, 0.4, 0.12)
    test_file = os.path.join(OUTPUT_DIR, "test_airfoil.dat")
    write_airfoil_file(test_file, x_u, y_u, x_l, y_l)

    pol = run_xfoil(test_file, REYNOLDS_NUMBER, aseq=ALPHA_BAND)
    if not pol:
        print("\n❌ ERROR: XFOIL test failed (no polar generated).")
        print("• Check XFOIL_EXE path")
        print("• Try running xfoil manually from a terminal")
        print("• Ensure .dat ordering: upper TE->LE then lower LE->TE")
        return

    valid = [(CL / CD, a, CL, CD) for (a, CL, CD, CM) in pol if CD > 0]
    if not valid:
        print("\n❌ ERROR: XFOIL produced no valid (CL, CD) in the AoA band.")
        return

    test_ld, test_a, cl_test, cd_test = max(valid, key=lambda z: z[0])
    print(f"✅ XFOIL OK. NACA 2412 best in band: α={test_a:.1f}°, "
          f"CL={cl_test:.3f}, CD={cd_test:.5f}, L/D={test_ld:.2f}\n")

    # --- Optimize
    print("Starting optimization...")
    print("-" * 70)
    start_time = datetime.now()

    result = differential_evolution(
        objective_function,
        bounds=BOUNDS,
        maxiter=OPTIMIZATION_ITERATIONS,
        popsize=POPULATION_SIZE,
        strategy="best1bin",
        mutation=F_MUT,
        recombination=CR,
        seed=RANDOM_SEED,
        disp=False,
        workers=-1,   # safe because run_xfoil uses unique temp dirs
        updating="deferred",
    )

    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    # --- Extract best from history (robust against noisy objectives)
    H = np.array(design_history)
    best_idx = np.argmax(H[:, 5]) if len(H) else -1
    if best_idx >= 0:
        best_mh, best_ph, best_th, best_cl, best_cd, best_ld = H[best_idx]
    else:
        best_mh, best_ph, best_th, best_cl, best_cd, best_ld = (np.nan,)*6

    # --- Report
    best_naca = f"{int(best_mh*100):02d}{int(best_ph*10):1d}{int(best_th*100):02d}"
    print()
    print("=" * 70)
    print("OPTIMIZATION COMPLETE")
    print("=" * 70)
    print(f"Time elapsed: {duration:.1f} s ({duration/60:.1f} min)")
    print(f"Total evaluations: {len(fitness_history)}")
    print(f"Successful evaluations: {np.sum(H[:, 5] > 0) if len(H) else 0}")
    print()
    print(f"Best observed airfoil: NACA {best_naca}")
    print(f"  m = {best_mh:.4f}  ({best_mh*100:.2f} %)")
    print(f"  p = {best_ph:.4f}  ({best_ph*100:.2f} % chord)")
    print(f"  t = {best_th:.4f}  ({best_th*100:.2f} %)")
    print()
    print("Performance (at α* giving max L/D in the band):")
    print(f"  CL = {best_cl:.4f}")
    print(f"  CD = {best_cd:.6f}")
    print(f"  L/D = {best_ld:.2f}")
    print()
    print(f"Improvement over NACA 2412: {((best_ld / test_ld) - 1) * 100:+.1f}%")
    print("=" * 70)

    # --- Save text results
    results_file = os.path.join(OUTPUT_DIR, "optimization_results.txt")
    with open(results_file, "w") as f:
        f.write("AIRFOIL OPTIMIZATION RESULTS\n")
        f.write("=" * 70 + "\n\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Reynolds Number: {REYNOLDS_NUMBER}\n")
        f.write(f"Ncrit: {NCRIT}  Mach: {MACH}\n")
        f.write(f"AoA band: {ALPHA_BAND}\n\n")
        f.write(f"Best Airfoil: NACA {best_naca}\n")
        f.write(f"  m = {best_mh:.4f}\n")
        f.write(f"  p = {best_ph:.4f}\n")
        f.write(f"  t = {best_th:.4f}\n\n")
        f.write("Performance (at α*):\n")
        f.write(f"  CL = {best_cl:.4f}\n")
        f.write(f"  CD = {best_cd:.6f}\n")
        f.write(f"  L/D = {best_ld:.2f}\n\n")
        f.write(f"Comparative L/D vs NACA 2412: {test_ld:.2f}\n")
        f.write(f"Improvement: {((best_ld / test_ld) - 1) * 100:+.1f}%\n")
        f.write(f"Evaluations: {len(fitness_history)}\n")
        f.write(f"Time: {duration:.1f} s\n")

    print(f"\nResults saved to {results_file}")

    # --- Plots
    print("\nGenerating plots...")
    plot_results(result, design_history, test_ld)

    print("\n✅ All done! Check the 'results' folder for outputs.")


if __name__ == "__main__":
    main()


AIRFOIL OPTIMIZATION WITH XFOIL (polar-based, AoA sweep)
Reynolds Number: 500000
Ncrit: 9 | Mach: 0.0
AoA band: -2.0..8.0 step 1.0 deg
Optimization: generations=120, popsize=15
Bounds: m(0.0, 0.09), p(0.3, 0.7), t(0.08, 0.18)

Testing XFOIL with NACA 2412...

❌ ERROR: XFOIL test failed (no polar generated).
• Check XFOIL_EXE path
• Try running xfoil manually from a terminal
• Ensure .dat ordering: upper TE->LE then lower LE->TE


In [11]:
# ============================== Airfoil L/D Optimization ==============================
# - NACA 4-digit generator with cosine spacing
# - XFOIL interface that writes a polar AND falls back to parsing stdout
# - Robust objective: max L/D over an AoA band
# - SciPy Differential Evolution optimizer
# - Plots + results saved even if later steps error
# ======================================================================================

import os
import re
import math
import shutil
import tempfile
import subprocess
from datetime import datetime
from functools import lru_cache

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution

# -------------------------------- CONFIG --------------------------------

# Set your XFOIL executable; Windows often needs a full path
# e.g., r"C:\Users\you\Desktop\xfoil\xfoil.exe"
XFOIL_EXE = "Downloads/XFOIL6.99/xfoil.exe"    # "xfoil" on Mac/Linux if in PATH

REYNOLDS_NUMBER = 500_000
NCRIT = 9
MACH = 0.0
MAX_ITER = 300
TIMEOUT = 120

# AoA sweep (robust objective)
ALPHA_BAND = (-2.0, 8.0, 1.0)   # a0, a1, da

# DE optimizer
OPTIMIZATION_ITERATIONS = 120   # generations
POPULATION_SIZE = 15            # SciPy meaning: popsize*d individuals
F_MUT = 0.7
CR = 0.9
RANDOM_SEED = 42

# Bounds for (m, p, t)  [fractions of chord]
BOUNDS = [
    (0.00, 0.09),   # m: max camber
    (0.30, 0.70),   # p: camber location
    (0.08, 0.18),   # t: max thickness
]

OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

BAD_FIT = 1e3        # finite penalty for failed evals
DEBUG_XFOIL = True  # set True to keep temp dir & save xfoil stdout/stderr

# ----------------------------- GEOMETRY ---------------------------------

def generate_naca_4digit(m, p, t, n_points=200):
    """Return (x_u,y_u,x_l,y_l) with cosine spacing."""
    beta = np.linspace(0.0, np.pi, n_points)
    x = 0.5 * (1 - np.cos(beta))  # cosine clustering near LE

    # Symmetric thickness distribution
    yt = 5.0 * t * (
        0.2969 * np.sqrt(np.clip(x, 1e-12, None))
        - 0.1260 * x
        - 0.3516 * x**2
        + 0.2843 * x**3
        - 0.1015 * x**4
    )

    yc = np.zeros_like(x)
    dyc = np.zeros_like(x)

    if m > 0 and 0 < p < 1:
        mask1 = x < p
        mask2 = ~mask1
        if p > 0:
            yc[mask1]  = m / p**2        * (2*p*x[mask1] - x[mask1]**2)
            dyc[mask1] = 2*m / p**2      * (p - x[mask1])
        if (1-p) > 0:
            yc[mask2]  = m / (1-p)**2    * ((1-2*p) + 2*p*x[mask2] - x[mask2]**2)
            dyc[mask2] = 2*m / (1-p)**2  * (p - x[mask2])

    theta = np.arctan(dyc)

    x_u = x - yt * np.sin(theta)
    y_u = yc + yt * np.cos(theta)
    x_l = x + yt * np.sin(theta)
    y_l = yc - yt * np.cos(theta)

    return x_u, y_u, x_l, y_l


def write_airfoil_file(filename, x_upper, y_upper, x_lower, y_lower):
    """XFOIL-friendly ordering: upper TE->LE then lower LE->TE."""
    with open(filename, "w") as f:
        f.write("NACA4\n")
        for i in range(len(x_upper)-1, -1, -1):               # upper TE -> LE
            f.write(f"{x_upper[i]:.6f} {y_upper[i]:.6f}\n")
        for i in range(len(x_lower)):                         # lower LE -> TE
            f.write(f"{x_lower[i]:.6f} {y_lower[i]:.6f}\n")

# ------------------------------ XFOIL I/O --------------------------------

def _parse_polar_file(polar_path):
    """Parse XFOIL polar -> list[(alpha, CL, CD, CM)]."""
    data = []
    if not os.path.exists(polar_path):
        return data
    with open(polar_path, "r") as f:
        for line in f:
            s = line.strip()
            if not s or s.startswith("#") or s.startswith("x"):
                continue
            sp = s.split()
            try:
                a  = float(sp[0]); CL = float(sp[1]); CD = float(sp[2])
                CM = float(sp[4]) if len(sp) > 4 else 0.0
                data.append((a, CL, CD, CM))
            except Exception:
                # defensive parse: numeric tokens only
                nums = []
                for tok in sp:
                    try: nums.append(float(tok))
                    except: pass
                if len(nums) >= 3:
                    a, CL, CD = nums[0], nums[1], nums[2]
                    CM = nums[4] if len(nums) > 4 else 0.0
                    data.append((a, CL, CD, CM))
    return data

# Regex for stdout lines like: "a = 4.000  CL = 0.6344  CD = 0.00470  Cm = -0.0229"
_STDOUT_LINE = re.compile(
    r"""a\s*=\s*([+\-]?\d+(?:\.\d*)?)      # alpha
        .*?CL\s*=\s*([+\-]?\d+\.?\d*(?:[Ee][+\-]?\d+)?) # CL
        .*?CD\s*=\s*([+\-]?\d+\.?\d*(?:[Ee][+\-]?\d+)?) # CD
        .*?(?:Cm|CM)\s*=\s*([+\-]?\d+\.?\d*(?:[Ee][+\-]?\d+)?)? # CM opt
    """,
    re.IGNORECASE | re.VERBOSE
)

def _parse_stdout_for_singlepoint(stdout_text):
    """Fallback: parse 'a=... CL=... CD=... Cm=...' lines. Returns last tuple or []."""
    rows = []
    for line in stdout_text.splitlines():
        m = _STDOUT_LINE.search(line)
        if m:
            a  = float(m.group(1))
            CL = float(m.group(2))
            CD = float(m.group(3))
            CM = float(m.group(4)) if m.group(4) is not None else 0.0
            rows.append((a, CL, CD, CM))
    return rows[-1:] if rows else []


def run_xfoil(airfoil_file, reynolds, aseq=None, alpha=None,
              ncrit=NCRIT, mach=MACH, max_iter=MAX_ITER, timeout=TIMEOUT):
    """
    XFOIL runner (Windows-friendly):
      - work in temp dir (no spaces)
      - CRLF line endings
      - exit VPAR submenu with blank line
      - use 'PACC out.pol' to avoid filename prompt
      - close polar, leave OPER, then QUIT
      - fallback to stdout parsing if no polar
    Returns list[(alpha, CL, CD, CM)].
    """
    tmpdir = tempfile.mkdtemp(prefix="xf_")
    keep_dir = DEBUG_XFOIL
    try:
        local_dat = os.path.join(tmpdir, "foil.dat")
        shutil.copyfile(airfoil_file, local_dat)
        polar_path = os.path.join(tmpdir, "out.pol")

        def build_script(iter_cap, use_seq=True):
            # IMPORTANT: blank line **after** VPAR to exit submenu
            lines = [
                "LOAD foil.dat",
                "PANE",
                "OPER",
                f"VISC {int(reynolds)}",
                "VPAR",
                f"N {int(ncrit)}",
                "",                 # <-- exit VPAR submenu back to OPER
                f"MACH {mach}",
                f"ITER {int(iter_cap)}",
                f"PACC {os.path.basename(polar_path)}",
            ]
            if use_seq and aseq is not None:
                a0, a1, da = aseq
                lines.append(f"ASEQ {a0} {a1} {da}")
            elif alpha is not None:
                lines.append(f"ALFA {alpha}")
            else:
                lines.append("ALFA 0.0")
            # Close polar, leave OPER, then QUIT
            lines += [
                "PACC",     # close polar accumulation
                "",         # (most builds: blank returns from OPER)
                "QUIT",
            ]
            return "\r\n".join(lines) + "\r\n"  # CRLF endings

        attempts = [(max_iter, True), (max_iter*2, True), (max_iter*2, False)]
        last_stdout = ""
        for iters, use_seq in attempts:
            script = build_script(iters, use_seq)
            cp = subprocess.run(
                [XFOIL_EXE],
                input=script,
                text=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=timeout,
                check=False,
                cwd=tmpdir,
            )
            last_stdout = cp.stdout
            if DEBUG_XFOIL:
                open(os.path.join(tmpdir, "xfoil_script.inp"), "w", newline="").write(script)
                open(os.path.join(tmpdir, "stdout.txt"), "w", newline="").write(cp.stdout)
                open(os.path.join(tmpdir, "stderr.txt"), "w", newline="").write(cp.stderr)
                print(f"[DEBUG] Temp dir: {tmpdir}")
                print("[DEBUG] XFOIL stdout tail:\n" + "\n".join(cp.stdout.splitlines()[-12:]))

            data = _parse_polar_file(polar_path)
            if data:
                return data

            sp = _parse_stdout_for_singlepoint(cp.stdout)
            if sp:
                return sp

        if DEBUG_XFOIL:
            print("[DEBUG] No polar and no parseable stdout. Last stdout tail:\n" +
                  "\n".join(last_stdout.splitlines()[-20:]))
        return []
    finally:
        if not keep_dir:
            shutil.rmtree(tmpdir, ignore_errors=True)



# --------------------------- OBJECTIVE & LOGS ----------------------------

eval_count = 0
best_fitness = -np.inf
fitness_history = []      # L/D per evaluation
design_history = []       # rows: [m, p, t, CL, CD, L/D]

@lru_cache(maxsize=4096)
def evaluate_tuple(m, p, t):
    """Return (best_ld, a*, CL, CD) for (m,p,t) over AoA band, else None."""
    x_u, y_u, x_l, y_l = generate_naca_4digit(m, p, t)
    dat_path = os.path.join(OUTPUT_DIR, f"foil_{hash((round(m,4),round(p,4),round(t,4)))}.dat")
    write_airfoil_file(dat_path, x_u, y_u, x_l, y_l)
    pol = run_xfoil(dat_path, REYNOLDS_NUMBER, aseq=ALPHA_BAND)
    if not pol:
        return None
    valid = [(CL/CD, a, CL, CD) for (a, CL, CD, CM) in pol if CD > 0]
    if not valid:
        return None
    return max(valid, key=lambda z: z[0])  # (ld, a*, CL, CD)

def objective_function(x):
    global eval_count, best_fitness
    eval_count += 1
    m, p, t = map(float, x)

    # Feasibility screen
    if not (0.00 <= m <= 0.09 and 0.30 <= p <= 0.70 and 0.08 <= t <= 0.18):
        fitness_history.append(0.0); design_history.append([m, p, t, 0.0, 0.0, 0.0])
        print(f"Eval {eval_count}: bounds reject m={m:.3f}, p={p:.3f}, t={t:.3f}")
        return BAD_FIT

    res = evaluate_tuple(round(m,4), round(p,4), round(t,4))
    if res is None:
        fitness_history.append(0.0); design_history.append([m, p, t, 0.0, 0.0, 0.0])
        print(f"Eval {eval_count}: XFOIL failed for m={m:.3f}, p={p:.3f}, t={t:.3f}")
        return BAD_FIT

    ld, a_star, CL, CD = res
    best_fitness = max(best_fitness, ld)
    fitness_history.append(ld)
    design_history.append([m, p, t, CL, CD, ld])

    tag = f"{int(m*100):02d}{int(p*10):1d}{int(t*100):02d}"
    print(f"Eval {eval_count}: NACA {tag:>4s} | α*={a_star:4.1f} | "
          f"CL={CL:6.3f} | CD={CD:7.4f} | L/D={ld:7.2f} | Best={best_fitness:7.2f}")
    return -ld

# ------------------------------ PLOTTING --------------------------------

def plot_results(result, design_history, test_ld):
    H = np.array(design_history)
    if len(H) == 0:
        print("(No design data to plot.)")
        return
    valid = H[:,5] > 0

    fig = plt.figure(figsize=(15,10))

    # 1) Convergence
    ax1 = plt.subplot(2,3,1)
    iters = np.arange(1, len(fitness_history)+1)
    ax1.plot(iters, fitness_history, alpha=0.35, label="All evals")
    ax1.plot(iters, np.maximum.accumulate(fitness_history), lw=2, label="Best so far")
    ax1.set_xlabel("Evaluation"); ax1.set_ylabel("L/D"); ax1.grid(alpha=0.3); ax1.legend()
    ax1.set_title("Optimization Convergence")

    # 2) Best geometry from optimizer result
    ax2 = plt.subplot(2,3,2)
    bm, bp, bt = result.x
    xu, yu, xl, yl = generate_naca_4digit(bm, bp, bt)
    ax2.plot(xu, yu, lw=2, label="Upper")
    ax2.plot(xl, yl, lw=2, label="Lower")
    ax2.set_aspect("equal", "box"); ax2.grid(alpha=0.3); ax2.legend()
    ax2.set_title(f"Best Airfoil: NACA {int(bm*100):02d}{int(bp*10):1d}{int(bt*100):02d}")
    ax2.set_xlabel("x/c"); ax2.set_ylabel("y/c")

    # 3) CL vs CD colored by L/D
    ax3 = plt.subplot(2,3,3)
    sc = ax3.scatter(H[valid,4], H[valid,3], c=H[valid,5], cmap="viridis", alpha=0.8)
    ax3.grid(alpha=0.3); ax3.set_xlabel("CD"); ax3.set_ylabel("CL")
    ax3.set_title("CL vs CD (color = L/D)")
    cbar = plt.colorbar(sc, ax=ax3); cbar.set_label("L/D")

    # 4) Design space: m vs t
    ax4 = plt.subplot(2,3,4)
    sc2 = ax4.scatter(H[valid,0]*100, H[valid,2]*100, c=H[valid,5], cmap="viridis", alpha=0.8)
    ax4.scatter(bm*100, bt*100, s=200, marker="*", edgecolors="k", zorder=5, label="Best")
    ax4.grid(alpha=0.3); ax4.legend()
    ax4.set_xlabel("Max camber m (%)"); ax4.set_ylabel("Max thickness t (%)")
    ax4.set_title("Design Space Exploration")

    # 5) L/D distribution
    ax5 = plt.subplot(2,3,5)
    ax5.hist(H[valid,5], bins=20, edgecolor="black", alpha=0.85)
    ax5.axvline(np.max(H[:,5]), color="r", ls="--", label="Best")
    if test_ld and test_ld > 0:
        ax5.axvline(test_ld, color="gray", ls=":", label="Baseline 2412")
    ax5.grid(alpha=0.3); ax5.legend()
    ax5.set_xlabel("L/D"); ax5.set_ylabel("Frequency"); ax5.set_title("L/D Distribution")

    # 6) Baseline vs optimized geometry
    ax6 = plt.subplot(2,3,6)
    xb_u, yb_u, xb_l, yb_l = generate_naca_4digit(0.02, 0.4, 0.12)
    ax6.plot(xb_u, yb_u, alpha=0.8, label="NACA 2412 baseline")
    ax6.plot(xb_l, yb_l, alpha=0.8)
    ax6.plot(xu, yu, lw=2, label="Optimized")
    ax6.plot(xl, yl, lw=2)
    ax6.set_aspect("equal", "box"); ax6.grid(alpha=0.3); ax6.legend()
    ax6.set_xlabel("x/c"); ax6.set_ylabel("y/c"); ax6.set_title("Baseline vs Optimized")

    plt.tight_layout()
    out_png = os.path.join(OUTPUT_DIR, "optimization_results.png")
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    print(f"\nPlots saved to {out_png}")
    plt.show()

# -------------------------------- MAIN -----------------------------------

def main():
    print("="*70)
    print("AIRFOIL OPTIMIZATION WITH XFOIL (polar/ stdout fallback, AoA sweep)")
    print("="*70)
    print(f"Reynolds Number: {REYNOLDS_NUMBER}")
    print(f"Ncrit: {NCRIT} | Mach: {MACH}")
    print(f"AoA band: {ALPHA_BAND[0]}..{ALPHA_BAND[1]} step {ALPHA_BAND[2]} deg")
    print(f"Optimization: generations={OPTIMIZATION_ITERATIONS}, popsize={POPULATION_SIZE}")
    print(f"Bounds: m{BOUNDS[0]}, p{BOUNDS[1]}, t{BOUNDS[2]}")
    print("="*70, "\n")

    result = None
    test_ld = None

    try:
        # Smoke test on NACA 2412
        print("Testing XFOIL with NACA 2412...")
        x_u, y_u, x_l, y_l = generate_naca_4digit(0.02, 0.4, 0.12)
        test_file = os.path.join(OUTPUT_DIR, "test_airfoil.dat")
        write_airfoil_file(test_file, x_u, y_u, x_l, y_l)

        pol = run_xfoil(test_file, REYNOLDS_NUMBER, aseq=ALPHA_BAND)
        if not pol:
            print("\n❌ ERROR: XFOIL test failed (no data parsed).")
            return

        valid = [(CL/CD, a, CL, CD) for (a, CL, CD, CM) in pol if CD > 0]
        if not valid:
            print("\n❌ ERROR: XFOIL produced no valid (CL, CD) in the AoA band.")
            return

        test_ld, test_a, cl_test, cd_test = max(valid, key=lambda z: z[0])
        print(f"✅ XFOIL OK. NACA 2412 best in band: α={test_a:.1f}°, "
              f"CL={cl_test:.3f}, CD={cd_test:.5f}, L/D={test_ld:.2f}\n")

        # Differential Evolution
        print("Starting optimization...")
        print("-"*70)
        start_time = datetime.now()

        result = differential_evolution(
            objective_function,
            bounds=BOUNDS,
            maxiter=OPTIMIZATION_ITERATIONS,
            popsize=POPULATION_SIZE,
            strategy="best1bin",
            mutation=F_MUT,
            recombination=CR,
            seed=RANDOM_SEED,
            disp=False,
            workers=-1,     # safe: each XFOIL call uses its own temp dir
            updating="deferred",
        )

        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()

        # Best observed from history (robust to noise)
        H = np.array(design_history)
        if len(H) and np.any(H[:,5] > 0):
            best_idx = np.argmax(H[:,5])
            best_mh, best_ph, best_th, best_cl, best_cd, best_ld = H[best_idx]
            best_naca = f"{int(best_mh*100):02d}{int(best_ph*10):1d}{int(best_th*100):02d}"

            print()
            print("="*70)
            print("OPTIMIZATION COMPLETE")
            print("="*70)
            print(f"Time elapsed: {duration:.1f} s ({duration/60:.1f} min)")
            print(f"Total evaluations: {len(fitness_history)}")
            print(f"Successful evaluations: {np.sum(H[:,5]>0)}")
            print()
            print(f"Best observed airfoil: NACA {best_naca}")
            print(f"  m = {best_mh:.4f}  ({best_mh*100:.2f} %)")
            print(f"  p = {best_ph:.4f}  ({best_ph*100:.2f} % chord)")
            print(f"  t = {best_th:.4f}  ({best_th*100:.2f} %)")
            print()
            print("Performance (at α* giving max L/D in the band):")
            print(f"  CL = {best_cl:.4f}")
            print(f"  CD = {best_cd:.6f}")
            print(f"  L/D = {best_ld:.2f}")
            if test_ld and test_ld > 0:
                print(f"\nImprovement over NACA 2412: {((best_ld/test_ld)-1)*100:+.1f}%")
            print("="*70)

            # Save text results
            results_file = os.path.join(OUTPUT_DIR, "optimization_results.txt")
            with open(results_file, "w") as f:
                f.write("AIRFOIL OPTIMIZATION RESULTS\n")
                f.write("="*70 + "\n\n")
                f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Reynolds Number: {REYNOLDS_NUMBER}\n")
                f.write(f"Ncrit: {NCRIT}  Mach: {MACH}\n")
                f.write(f"AoA band: {ALPHA_BAND}\n\n")
                f.write(f"Best Airfoil: NACA {best_naca}\n")
                f.write(f"  m = {best_mh:.4f}\n")
                f.write(f"  p = {best_ph:.4f}\n")
                f.write(f"  t = {best_th:.4f}\n\n")
                f.write("Performance (at α*):\n")
                f.write(f"  CL = {best_cl:.4f}\n")
                f.write(f"  CD = {best_cd:.6f}\n")
                f.write(f"  L/D = {best_ld:.2f}\n\n")
                f.write(f"Comparative L/D vs NACA 2412: {test_ld:.2f}\n")
                if test_ld and test_ld > 0:
                    f.write(f"Improvement: {((best_ld/test_ld)-1)*100:+.1f}%\n")
                f.write(f"Evaluations: {len(fitness_history)}\n")
                f.write(f"Time: {duration:.1f} s\n")
            print(f"\nResults saved to {results_file}")

        else:
            print("\n⚠️ Optimization finished but no valid evaluations were recorded.")

    except Exception as e:
        print(f"\n⚠️ Runtime error: {e}")

    finally:
        # Always try to plot whatever we have
        try:
            if test_ld is None or not np.isfinite(test_ld):
                test_ld = max([row[5] for row in design_history if row[5] > 0], default=0.0)
            class _R: pass
            if result is None:
                # fallback: fake minimal result using best history x
                H = np.array(design_history)
                if len(H) and np.any(H[:,5] > 0):
                    _r = _R(); _r.x = H[np.argmax(H[:,5])][:3]
                    result = _r
            if result is not None and len(design_history) > 0:
                print("\nGenerating plots...")
                plot_results(result, design_history, test_ld)
        except Exception as pe:
            print(f"Plotting failed: {pe}")


if __name__ == "__main__":
    main()


AIRFOIL OPTIMIZATION WITH XFOIL (polar/ stdout fallback, AoA sweep)
Reynolds Number: 500000
Ncrit: 9 | Mach: 0.0
AoA band: -2.0..8.0 step 1.0 deg
Optimization: generations=120, popsize=15
Bounds: m(0.0, 0.09), p(0.3, 0.7), t(0.08, 0.18)

Testing XFOIL with NACA 2412...
[DEBUG] Temp dir: C:\Users\User\AppData\Local\Temp\xf_vmqdvczd
[DEBUG] XFOIL stdout tail:
 Airfoil archived with polar: NACA4                                           

Enter  polar save filename  OR  <return> for no file   s>  
 New polar save file available

Enter  polar dump filename  OR  <return> for no file   s>  
 New polar dump file available

 Polar accumulation enabled

.OPERva   c>  
 XFOIL   c>  
[DEBUG] Temp dir: C:\Users\User\AppData\Local\Temp\xf_vmqdvczd
[DEBUG] XFOIL stdout tail:
   alpha    CL        CD       CDp       CM     Top_Xtr  Bot_Xtr
  ------ -------- --------- --------- -------- -------- --------

 Old polar save file available for appending

Enter  polar dump filename  OR  <return> for no fil

In [12]:
import numpy as np
import subprocess
import os
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
from datetime import datetime

# =============================================================================
# CONFIGURATION
# =============================================================================

# XFOIL settings
XFOIL_EXE = "Downloads/XFOIL6.99/xfoil.exe"  # Change to "xfoil" on Mac/Linux
REYNOLDS_NUMBER = 500000
ANGLE_OF_ATTACK = 4.0
MAX_ITER = 100  # XFOIL iterations

# Optimization settings
OPTIMIZATION_ITERATIONS = 100  # Start with 100, increase to 200 later
POPULATION_SIZE = 15

# Design variable bounds (NACA 4-digit)
# M: maximum camber (0-9% of chord)
# P: position of maximum camber (10-90% of chord)
# T: maximum thickness (6-21% of chord)
BOUNDS = [
    (0.00, 0.09),  # M: 0-9% camber
    (0.10, 0.90),  # P: 10-90% position
    (0.06, 0.21),  # T: 6-21% thickness
]

# Output directory
OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =============================================================================
# NACA 4-DIGIT AIRFOIL GENERATOR
# =============================================================================

def generate_naca_4digit(m, p, t, n_points=200):
    """
    Generate NACA 4-digit airfoil coordinates.
    
    Parameters:
    -----------
    m : float
        Maximum camber (as fraction of chord, e.g., 0.02 for 2%)
    p : float
        Position of maximum camber (as fraction of chord, e.g., 0.4 for 40%)
    t : float
        Maximum thickness (as fraction of chord, e.g., 0.12 for 12%)
    n_points : int
        Number of points to generate
    
    Returns:
    --------
    x, y_upper, y_lower : arrays
        Airfoil coordinates
    """
    # Cosine spacing for better resolution near leading edge
    beta = np.linspace(0, np.pi, n_points)
    x = 0.5 * (1 - np.cos(beta))
    
    # Thickness distribution (symmetric)
    yt = 5 * t * (0.2969 * np.sqrt(x) - 0.1260 * x - 0.3516 * x**2 + 
                   0.2843 * x**3 - 0.1015 * x**4)
    
    # Mean camber line
    yc = np.zeros_like(x)
    dyc_dx = np.zeros_like(x)
    
    if m > 0 and p > 0:  # Cambered airfoil
        # Forward of maximum camber
        mask1 = x < p
        yc[mask1] = m / p**2 * (2 * p * x[mask1] - x[mask1]**2)
        dyc_dx[mask1] = 2 * m / p**2 * (p - x[mask1])
        
        # Aft of maximum camber
        mask2 = x >= p
        yc[mask2] = m / (1 - p)**2 * ((1 - 2 * p) + 2 * p * x[mask2] - x[mask2]**2)
        dyc_dx[mask2] = 2 * m / (1 - p)**2 * (p - x[mask2])
    
    # Angle of camber line
    theta = np.arctan(dyc_dx)
    
    # Upper and lower surface coordinates
    x_upper = x - yt * np.sin(theta)
    y_upper = yc + yt * np.cos(theta)
    
    x_lower = x + yt * np.sin(theta)
    y_lower = yc - yt * np.cos(theta)
    
    return x_upper, y_upper, x_lower, y_lower

def write_airfoil_file(filename, x_upper, y_upper, x_lower, y_lower):
    """Write airfoil coordinates to file in XFOIL format."""
    with open(filename, 'w') as f:
        f.write("NACA Airfoil\n")
        # Upper surface (leading edge to trailing edge)
        for i in range(len(x_upper) - 1, -1, -1):
            f.write(f"  {x_upper[i]:.6f}  {y_upper[i]:.6f}\n")
        # Lower surface (leading edge to trailing edge)
        for i in range(len(x_lower)):
            f.write(f"  {x_lower[i]:.6f}  {y_lower[i]:.6f}\n")

# =============================================================================
# XFOIL INTERFACE
# =============================================================================

def run_xfoil(airfoil_file, reynolds, alpha, max_iter=100):
    """
    Run XFOIL and extract Cl and Cd.
    
    Parameters:
    -----------
    airfoil_file : str
        Path to airfoil coordinate file
    reynolds : float
        Reynolds number
    alpha : float
        Angle of attack (degrees)
    max_iter : int
        Maximum XFOIL iterations
    
    Returns:
    --------
    cl, cd : float or None
        Lift and drag coefficients (None if failed)
    """
    
    # Create XFOIL command file
    commands = f"""LOAD {airfoil_file}
OPER
VISC {reynolds}
ITER {max_iter}
ALFA {alpha}
QUIT
"""
    
    cmd_file = os.path.join(OUTPUT_DIR, "xfoil_commands.txt")
    with open(cmd_file, 'w') as f:
        f.write(commands)
    
    try:
        # Run XFOIL with timeout
        result = subprocess.run(
            [XFOIL_EXE],
            stdin=open(cmd_file, 'r'),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=30,
            text=True
        )
        
        output = result.stdout
        
        # Parse output for Cl and Cd
        cl, cd = None, None
        for line in output.split('\n'):
            if 'CL =' in line:
                try:
                    parts = line.split()
                    cl_idx = parts.index('CL')
                    cd_idx = parts.index('CD')
                    cl = float(parts[cl_idx + 2])
                    cd = float(parts[cd_idx + 2])
                    break
                except (ValueError, IndexError):
                    continue
        
        return cl, cd
    
    except (subprocess.TimeoutExpired, FileNotFoundError, Exception) as e:
        print(f"XFOIL error: {e}")
        return None, None

# =============================================================================
# OPTIMIZATION
# =============================================================================

# Global variables to track optimization progress
eval_count = 0
best_fitness = -np.inf
fitness_history = []
design_history = []

def objective_function(x):
    """
    Objective function for optimization.
    
    Parameters:
    -----------
    x : array [m, p, t]
        Design variables
    
    Returns:
    --------
    fitness : float
        Negative L/D ratio (we minimize, so negate for maximization)
    """
    global eval_count, best_fitness
    eval_count += 1
    
    m, p, t = x
    
    # Generate airfoil
    try:
        x_upper, y_upper, x_lower, y_lower = generate_naca_4digit(m, p, t)
        
        # Write to file
        airfoil_file = os.path.join(OUTPUT_DIR, f"airfoil_{eval_count}.dat")
        write_airfoil_file(airfoil_file, x_upper, y_upper, x_lower, y_lower)
        
        # Run XFOIL
        cl, cd = run_xfoil(airfoil_file, REYNOLDS_NUMBER, ANGLE_OF_ATTACK, MAX_ITER)
        
        # Calculate fitness
        if cl is not None and cd is not None and cd > 0:
            ld_ratio = cl / cd
            fitness = -ld_ratio  # Negative because we minimize
            
            # Track best
            if ld_ratio > best_fitness:
                best_fitness = ld_ratio
            
            # Store history
            fitness_history.append(ld_ratio)
            design_history.append([m, p, t, cl, cd, ld_ratio])
            
            # Print progress
            naca_designation = f"{int(m*100)}{int(p*10)}{int(t*100)}"
            print(f"Eval {eval_count}: NACA {naca_designation:4s} | "
                  f"Cl={cl:6.3f} | Cd={cd:7.4f} | L/D={ld_ratio:7.2f} | "
                  f"Best L/D={best_fitness:7.2f}")
            
            return fitness
        else:
            # XFOIL failed - return bad fitness
            fitness_history.append(0)
            design_history.append([m, p, t, 0, 0, 0])
            print(f"Eval {eval_count}: XFOIL failed for M={m:.3f}, P={p:.2f}, T={t:.3f}")
            return 1000  # Large penalty
    
    except Exception as e:
        print(f"Eval {eval_count}: Error - {e}")
        fitness_history.append(0)
        design_history.append([m, p, t, 0, 0, 0])
        return 1000

# =============================================================================
# VISUALIZATION
# =============================================================================

def plot_results(result, design_history):
    """Generate all result plots."""
    
    # Convert history to array
    history = np.array(design_history)
    
    # Create figure with subplots
    fig = plt.figure(figsize=(15, 10))
    
    # 1. Convergence history
    ax1 = plt.subplot(2, 3, 1)
    iterations = range(1, len(fitness_history) + 1)
    ax1.plot(iterations, fitness_history, 'b-', alpha=0.3, label='All evaluations')
    
    # Running best
    running_best = [max(fitness_history[:i+1]) for i in range(len(fitness_history))]
    ax1.plot(iterations, running_best, 'r-', linewidth=2, label='Best so far')
    
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('L/D Ratio')
    ax1.set_title('Optimization Convergence')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Best airfoil shape
    ax2 = plt.subplot(2, 3, 2)
    best_m, best_p, best_t = result.x
    x_u, y_u, x_l, y_l = generate_naca_4digit(best_m, best_p, best_t)
    
    ax2.plot(x_u, y_u, 'b-', linewidth=2, label='Upper surface')
    ax2.plot(x_l, y_l, 'r-', linewidth=2, label='Lower surface')
    ax2.set_xlabel('x/c')
    ax2.set_ylabel('y/c')
    ax2.set_title(f'Best Airfoil: NACA {int(best_m*100)}{int(best_p*10)}{int(best_t*100)}')
    ax2.axis('equal')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # 3. Cl vs Cd scatter
    ax3 = plt.subplot(2, 3, 3)
    valid_mask = history[:, 3] > 0  # Where Cl > 0 (valid runs)
    ax3.scatter(history[valid_mask, 4], history[valid_mask, 3], 
                c=history[valid_mask, 5], cmap='viridis', alpha=0.6)
    ax3.set_xlabel('Cd (Drag Coefficient)')
    ax3.set_ylabel('Cl (Lift Coefficient)')
    ax3.set_title('Cl vs Cd (colored by L/D)')
    ax3.grid(True, alpha=0.3)
    cbar = plt.colorbar(ax3.collections[0], ax=ax3)
    cbar.set_label('L/D Ratio')
    
    # 4. Design space exploration - M vs T
    ax4 = plt.subplot(2, 3, 4)
    scatter = ax4.scatter(history[valid_mask, 0]*100, history[valid_mask, 2]*100,
                         c=history[valid_mask, 5], cmap='viridis', alpha=0.6)
    ax4.scatter(best_m*100, best_t*100, color='red', s=200, marker='*', 
                edgecolors='black', linewidths=2, label='Best', zorder=5)
    ax4.set_xlabel('Maximum Camber (%)')
    ax4.set_ylabel('Maximum Thickness (%)')
    ax4.set_title('Design Space Exploration')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Parameter distributions
    ax5 = plt.subplot(2, 3, 5)
    ax5.hist(history[valid_mask, 5], bins=20, alpha=0.7, edgecolor='black')
    ax5.axvline(best_fitness, color='red', linestyle='--', linewidth=2, label='Best')
    ax5.set_xlabel('L/D Ratio')
    ax5.set_ylabel('Frequency')
    ax5.set_title('L/D Distribution')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. Comparison with initial design
    ax6 = plt.subplot(2, 3, 6)
    # Use NACA 2412 as baseline
    x_u_base, y_u_base, x_l_base, y_l_base = generate_naca_4digit(0.02, 0.4, 0.12)
    
    ax6.plot(x_u_base, y_u_base, 'gray', linewidth=1.5, label='NACA 2412 (baseline)', alpha=0.7)
    ax6.plot(x_l_base, y_l_base, 'gray', linewidth=1.5, alpha=0.7)
    ax6.plot(x_u, y_u, 'b-', linewidth=2, label='Optimized')
    ax6.plot(x_l, y_l, 'b-', linewidth=2)
    ax6.set_xlabel('x/c')
    ax6.set_ylabel('y/c')
    ax6.set_title('Baseline vs Optimized')
    ax6.axis('equal')
    ax6.grid(True, alpha=0.3)
    ax6.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'optimization_results.png'), dpi=300, bbox_inches='tight')
    print(f"\nPlots saved to {OUTPUT_DIR}/optimization_results.png")
    plt.show()

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main optimization routine."""
    
    print("="*70)
    print("AIRFOIL OPTIMIZATION WITH XFOIL")
    print("="*70)
    print(f"Reynolds Number: {REYNOLDS_NUMBER}")
    print(f"Angle of Attack: {ANGLE_OF_ATTACK}°")
    print(f"Optimization Iterations: {OPTIMIZATION_ITERATIONS}")
    print(f"Population Size: {POPULATION_SIZE}")
    print(f"Design Variables: M (camber), P (position), T (thickness)")
    print(f"Objective: Maximize L/D ratio")
    print("="*70)
    print()
    
    # Test XFOIL first
    print("Testing XFOIL with NACA 2412...")
    x_u, y_u, x_l, y_l = generate_naca_4digit(0.02, 0.4, 0.12)
    test_file = os.path.join(OUTPUT_DIR, "test_airfoil.dat")
    write_airfoil_file(test_file, x_u, y_u, x_l, y_l)
    cl_test, cd_test = run_xfoil(test_file, REYNOLDS_NUMBER, ANGLE_OF_ATTACK, MAX_ITER)
    
    if cl_test is None:
        print("\n❌ ERROR: XFOIL test failed!")
        print("Make sure xfoil.exe is in the same folder or in your PATH")
        return
    else:
        print(f"✅ XFOIL working! NACA 2412: Cl={cl_test:.3f}, Cd={cd_test:.5f}, L/D={cl_test/cd_test:.2f}")
        print()
    
    # Run optimization
    print("Starting optimization...")
    print("-"*70)
    
    start_time = datetime.now()
    
    result = differential_evolution(
        objective_function,
        BOUNDS,
        maxiter=OPTIMIZATION_ITERATIONS,
        popsize=POPULATION_SIZE,
        strategy='best1bin',
        seed=42,
        disp=False,
        workers=1  # XFOIL can't run in parallel easily
    )
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    # Extract best design
    best_m, best_p, best_t = result.x
    best_naca = f"{int(best_m*100)}{int(best_p*10)}{int(best_t*100)}"
    
    # Get best performance
    history = np.array(design_history)
    best_idx = np.argmax(history[:, 5])
    best_cl = history[best_idx, 3]
    best_cd = history[best_idx, 4]
    best_ld = history[best_idx, 5]
    
    # Print results
    print()
    print("="*70)
    print("OPTIMIZATION COMPLETE!")
    print("="*70)
    print(f"Time elapsed: {duration:.1f} seconds ({duration/60:.1f} minutes)")
    print(f"Total evaluations: {eval_count}")
    print(f"Successful evaluations: {np.sum(history[:, 5] > 0)}")
    print()
    print(f"Best Airfoil: NACA {best_naca}")
    print(f"  M (camber) = {best_m:.4f} ({best_m*100:.2f}%)")
    print(f"  P (position) = {best_p:.4f} ({best_p*100:.1f}%)")
    print(f"  T (thickness) = {best_t:.4f} ({best_t*100:.2f}%)")
    print()
    print(f"Performance:")
    print(f"  Cl (lift)  = {best_cl:.4f}")
    print(f"  Cd (drag)  = {best_cd:.6f}")
    print(f"  L/D ratio  = {best_ld:.2f}")
    print()
    print(f"Improvement over NACA 2412: {((best_ld/(cl_test/cd_test))-1)*100:+.1f}%")
    print("="*70)
    
    # Save results to file
    results_file = os.path.join(OUTPUT_DIR, "optimization_results.txt")
    with open(results_file, 'w') as f:
        f.write("AIRFOIL OPTIMIZATION RESULTS\n")
        f.write("="*70 + "\n\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Reynolds Number: {REYNOLDS_NUMBER}\n")
        f.write(f"Angle of Attack: {ANGLE_OF_ATTACK}°\n\n")
        f.write(f"Best Airfoil: NACA {best_naca}\n")
        f.write(f"  M = {best_m:.4f}\n")
        f.write(f"  P = {best_p:.4f}\n")
        f.write(f"  T = {best_t:.4f}\n\n")
        f.write(f"Performance:\n")
        f.write(f"  Cl = {best_cl:.4f}\n")
        f.write(f"  Cd = {best_cd:.6f}\n")
        f.write(f"  L/D = {best_ld:.2f}\n\n")
        f.write(f"Evaluations: {eval_count}\n")
        f.write(f"Time: {duration:.1f} seconds\n")
    
    print(f"\nResults saved to {results_file}")
    
    # Generate plots
    print("\nGenerating plots...")
    plot_results(result, design_history)
    
    print("\n✅ All done! Check the 'results' folder for outputs.")

if __name__ == "__main__":
    main()

AIRFOIL OPTIMIZATION WITH XFOIL
Reynolds Number: 500000
Angle of Attack: 4.0°
Optimization Iterations: 100
Population Size: 15
Design Variables: M (camber), P (position), T (thickness)
Objective: Maximize L/D ratio

Testing XFOIL with NACA 2412...

❌ ERROR: XFOIL test failed!
Make sure xfoil.exe is in the same folder or in your PATH


In [19]:
# ---------------------- simple_xfoil_parse_test.py ----------------------
import os, re, shutil, tempfile, subprocess
import numpy as np

# --- CONFIG -------------------------------------------------------------
XFOIL_EXE = r"Downloads/XFOIL6.99/xfoil.exe"  # <-- set your path
RE = 500_000
NCRIT = 9
MACH = 0.0
ALPHA = 4.0             # single-point test
ALPHA_SWEEP = None      # e.g. (-2, 6, 2) to test ASEQ; or leave None for single alpha
MAX_ITER = 200
TIMEOUT = 60
KEEP_TEMP = True        # True = keep temp dir so you can inspect stdout.txt
# -----------------------------------------------------------------------

# --- Simple NACA 4-digit generator (2412 baseline used below) ----------
def naca_4digit_coords(m=0.02, p=0.4, t=0.12, n=200):
    beta = np.linspace(0.0, np.pi, n)
    x = 0.5*(1-np.cos(beta))
    yt = 5.0*t*(0.2969*np.sqrt(np.clip(x,1e-12,None)) - 0.1260*x
                -0.3516*x**2 + 0.2843*x**3 - 0.1015*x**4)
    yc = np.zeros_like(x); dyc = np.zeros_like(x)
    if m > 0 and 0 < p < 1:
        m1 = x < p
        m2 = ~m1
        yc[m1]  = m/p**2*(2*p*x[m1]-x[m1]**2)
        dyc[m1] = 2*m/p**2*(p-x[m1])
        yc[m2]  = m/(1-p)**2*((1-2*p)+2*p*x[m2]-x[m2]**2)
        dyc[m2] = 2*m/(1-p)**2*(p-x[m2])
    th = np.arctan(dyc)
    xu = x - yt*np.sin(th); yu = yc + yt*np.cos(th)
    xl = x + yt*np.sin(th); yl = yc - yt*np.cos(th)
    return xu, yu, xl, yl

def write_xfoil_dat(path, xu,yu,xl,yl):
    with open(path, "w") as f:
        f.write("NACA4\n")
        for i in range(len(xu)-1, -1, -1):  # upper TE->LE
            f.write(f"{xu[i]:.6f} {yu[i]:.6f}\n")
        for i in range(len(xl)):            # lower LE->TE
            f.write(f"{xl[i]:.6f} {yl[i]:.6f}\n")

# --- Parsers ------------------------------------------------------------
def parse_polar_file(path):
    """Return list[(alpha, CL, CD, CM)] from out.pol; [] if none."""
    rows = []
    if not os.path.exists(path): return rows
    with open(path) as f:
        for line in f:
            s = line.strip()
            if not s or s.startswith("#") or s.startswith("x"):  # skip headers
                continue
            parts = s.split()
            try:
                a  = float(parts[0]); CL = float(parts[1]); CD = float(parts[2])
                CM = float(parts[4]) if len(parts) > 4 else 0.0
                rows.append((a,CL,CD,CM))
            except:
                # numeric-only fallback
                nums = []
                for tok in parts:
                    try: nums.append(float(tok))
                    except: pass
                if len(nums) >= 3:
                    a,CL,CD = nums[0], nums[1], nums[2]
                    CM = nums[4] if len(nums) > 4 else 0.0
                    rows.append((a,CL,CD,CM))
    return rows

# two-line tolerant stdout parsing:
_RE_A_CL = re.compile(r"a\s*=\s*([+-]?\d+(?:\.\d*)?)\s+.*?CL\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CD   = re.compile(r"\bCD\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CM   = re.compile(r"\bC[Mm]\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)

def parse_stdout_for_singlepoint(stdout_text):
    """Stitches 'a & CL' (line 1) with 'Cm & CD' (same/next line). Returns last record if any."""
    rows = []
    cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    for line in stdout_text.splitlines():
        m1 = _RE_A_CL.search(line)
        if m1:
            # flush previous if complete
            if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
                rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
                cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
            cur["a"]  = float(m1.group(1))
            cur["CL"] = float(m1.group(2))
            continue
        m_cd = _RE_CD.search(line)
        if m_cd: 
            try: cur["CD"] = float(m_cd.group(1))
            except: pass
        m_cm = _RE_CM.search(line)
        if m_cm:
            try: cur["CM"] = float(m_cm.group(1))
            except: pass
        if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
            rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
            cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
        rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
    return rows[-1:] if rows else []

# --- XFOIL runner (Windows-friendly, tiny) ------------------------------
def run_xfoil_once(dat_file, re_num=RE, ncrit=NCRIT, mach=MACH,
                   alpha=ALPHA, aseq=ALPHA_SWEEP, max_iter=MAX_ITER, timeout=TIMEOUT):
    """
    Returns list[(alpha, CL, CD, CM)] from polar or stdout.
    Uses CRLF line endings, exits VPAR submenu, writes polar without prompts.
    """
    tmp = tempfile.mkdtemp(prefix="xf_test_")
    try:
        # work inside tmp to avoid spaces in paths
        local_dat = os.path.join(tmp, "foil.dat")
        shutil.copyfile(dat_file, local_dat)
        polar_name = "out.pol"
        polar_path = os.path.join(tmp, polar_name)

        # build script lines (CRLF)
        lines = [
                    f"LOAD {local_dat}",
                    "PANE",
                    "OPER",
                    f"VISC {int(re_num)}",
                    "VPAR",
                    f"N {int(ncrit)}",
                    "",      # <- this blank line exits VPAR menu
                    f"MACH {mach}",
                    f"ITER {int(max_iter)}",
                    "PACC out.pol",
                ]
        if aseq:
                a0, a1, da = aseq
                lines.append(f"ASEQ {a0} {a1} {da}")
        else:
                lines.append(f"ALFA {alpha}")
                lines += [
                "PACC",   # CLOSE polar file
                "",
                "QUIT",
                ]
        script = "\r\n".join(lines) + "\r\n"


        cp = subprocess.run(
            [XFOIL_EXE],
            input=script,
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=timeout,
            cwd=tmp,
            check=False,
        )

        # Save for debugging
        if KEEP_TEMP:
            with open(os.path.join(tmp, "xfoil_script.inp"), "w", newline="") as f: f.write(script)
            with open(os.path.join(tmp, "stdout.txt"), "w", newline="") as f: f.write(cp.stdout)
            with open(os.path.join(tmp, "stderr.txt"), "w", newline="") as f: f.write(cp.stderr)
            print("[DEBUG] Temp dir:", tmp)
            print("[DEBUG] XFOIL stdout tail:\n", "\n".join(cp.stdout.splitlines()[-12:]))

        # 1) Try polar
        rows = parse_polar_file(polar_path)
        if rows:
            return rows

        # 2) Fallback to stdout
        sp = parse_stdout_for_singlepoint(cp.stdout)
        return sp

    finally:
        if not KEEP_TEMP:
            shutil.rmtree(tmp, ignore_errors=True)

# --- Main: single-airfoil test ------------------------------------------
if __name__ == "__main__":
    # Make a test .dat for NACA 2412
    xu,yu,xl,yl = naca_4digit_coords(m=0.02, p=0.4, t=0.12, n=200)
    dat_path = os.path.abspath("naca2412_test.dat")
    write_xfoil_dat(dat_path, xu,yu,xl,yl)
    print("Wrote:", dat_path)

    # Run XFOIL once
    results = run_xfoil_once(dat_path)

    if not results:
        print("\n❌ No data parsed (neither polar nor stdout). Check temp stdout.txt (see above).")
    else:
        print("\n✅ Parsed data:")
        for (a,CL,CD,CM) in results:
            print(f"  alpha={a:6.2f} deg | CL={CL:7.4f} | CD={CD:9.6f} | CM={CM:7.4f}")
        # If single-point, list has one row; if sweep, it may have several.
# -----------------------------------------------------------------------


Wrote: C:\Users\User\naca2412_test.dat
[DEBUG] Temp dir: C:\Users\User\AppData\Local\Temp\xf_test_j4lo2h5l
[DEBUG] XFOIL stdout tail:
  Airfoil archived with polar: NACA4                                           

Enter  polar save filename  OR  <return> for no file   s>  
 New polar save file available

Enter  polar dump filename  OR  <return> for no file   s>  
 New polar dump file available

 Polar accumulation enabled

.OPERva   c>  
 XFOIL   c>  

❌ No data parsed (neither polar nor stdout). Check temp stdout.txt (see above).


In [16]:
s = "\r\n"
print(repr(s))

'\r\n'


In [20]:
# ---------------------- simple_xfoil_parse_test.py ----------------------
import os, re, shutil, tempfile, subprocess
import numpy as np

# ======================= CONFIG =========================================
XFOIL_EXE = r"Downloads/XFOIL6.99/xfoil.exe"  # <-- set your path
RE = 500_000
NCRIT = 9
MACH = 0.0
ALPHA = 4.0             # single-point test
ALPHA_SWEEP = None      # e.g. (-2, 6, 2) to test ASEQ; keep None for single alpha
MAX_ITER = 200
TIMEOUT = 60
KEEP_TEMP = True        # keep temp dir so you can inspect stdout.txt/script
# =======================================================================

# ---------------------- NACA 4-digit -----------------------------------
def naca_4digit_coords(m=0.02, p=0.4, t=0.12, n=200):
    beta = np.linspace(0.0, np.pi, n)
    x = 0.5*(1 - np.cos(beta))  # cosine spacing
    yt = 5.0*t*(0.2969*np.sqrt(np.clip(x,1e-12,None)) - 0.1260*x
                -0.3516*x**2 + 0.2843*x**3 - 0.1015*x**4)
    yc = np.zeros_like(x); dyc = np.zeros_like(x)
    if m > 0 and 0 < p < 1:
        m1 = x < p
        m2 = ~m1
        yc[m1]  = m/p**2*(2*p*x[m1] - x[m1]**2)
        dyc[m1] = 2*m/p**2*(p - x[m1])
        yc[m2]  = m/(1-p)**2*((1-2*p) + 2*p*x[m2] - x[m2]**2)
        dyc[m2] = 2*m/(1-p)**2*(p - x[m2])
    th = np.arctan(dyc)
    xu = x - yt*np.sin(th); yu = yc + yt*np.cos(th)
    xl = x + yt*np.sin(th); yl = yc - yt*np.cos(th)
    return xu, yu, xl, yl

def write_xfoil_dat(path, xu,yu,xl,yl):
    with open(path, "w") as f:
        f.write("NACA4\n")
        # Upper TE->LE
        for i in range(len(xu)-1, -1, -1):
            f.write(f"{xu[i]:.6f} {yu[i]:.6f}\n")
        # Lower LE->TE
        for i in range(len(xl)):
            f.write(f"{xl[i]:.6f} {yl[i]:.6f}\n")

# ---------------------- Parsers ----------------------------------------
def parse_polar_file(path):
    """Return list[(alpha, CL, CD, CM)] from polar file; [] if none."""
    rows = []
    if not os.path.exists(path):
        return rows
    with open(path) as f:
        for line in f:
            s = line.strip()
            if not s or s.startswith("#") or s.startswith("x"):
                continue
            parts = s.split()
            try:
                a  = float(parts[0]); CL = float(parts[1]); CD = float(parts[2])
                CM = float(parts[4]) if len(parts) > 4 else 0.0
                rows.append((a,CL,CD,CM))
            except:
                nums = []
                for tok in parts:
                    try: nums.append(float(tok))
                    except: pass
                if len(nums) >= 3:
                    a,CL,CD = nums[0], nums[1], nums[2]
                    CM = nums[4] if len(nums) > 4 else 0.0
                    rows.append((a,CL,CD,CM))
    return rows

# tolerant stdout regex (two-line format: 'a & CL' then possibly next line 'Cm & CD')
_RE_A_CL = re.compile(r"a\s*=\s*([+-]?\d+(?:\.\d*)?)\s+.*?CL\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CD   = re.compile(r"\bCD\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CM   = re.compile(r"\bC[Mm]\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)

def parse_stdout_for_singlepoint(stdout_text):
    """Stitches 'a & CL' with 'Cm & CD' (next line OK). Returns last complete record; [] if none."""
    rows = []
    cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    for line in stdout_text.splitlines():
        m1 = _RE_A_CL.search(line)
        if m1:
            if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
                rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
                cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
            cur["a"]  = float(m1.group(1))
            cur["CL"] = float(m1.group(2))
            continue
        m_cd = _RE_CD.search(line)
        if m_cd:
            try: cur["CD"] = float(m_cd.group(1))
            except: pass
        m_cm = _RE_CM.search(line)
        if m_cm:
            try: cur["CM"] = float(m_cm.group(1))
            except: pass
        if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
            rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
            cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
        rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
    return rows[-1:] if rows else []

# ---------------------- XFOIL runner (Windows-friendly) ----------------
def run_xfoil_once(dat_file, re_num=RE, ncrit=NCRIT, mach=MACH,
                   alpha=ALPHA, aseq=ALPHA_SWEEP, max_iter=MAX_ITER, timeout=TIMEOUT):
    """
    Returns list[(alpha, CL, CD, CM)].
    - works in a temp dir (paths w/o spaces)
    - CRLF line endings
    - VPAR submenu exit via blank line
    - 'PACC out.pol' + blank line to skip dump filename prompt
    - close PACC, leave OPER, QUIT
    """
    tmp = tempfile.mkdtemp(prefix="xf_test_")
    try:
        local_dat = os.path.join(tmp, "foil.dat")
        shutil.copyfile(dat_file, local_dat)
        polar_name = "out.pol"
        polar_path = os.path.join(tmp, polar_name)

        lines = [
            "LOAD foil.dat",
            "PANE",
            "OPER",
            f"VISC {int(re_num)}",
            "VPAR",
            f"N {int(ncrit)}",
            "",                   # exit VPAR submenu back to OPER
            f"MACH {mach}",
            f"ITER {int(max_iter)}",
            f"PACC {polar_name}", # open polar (no prompt for filename)
            "",                   # <-- hit <return> to SKIP dump filename prompt
        ]
        if aseq:
            a0,a1,da = aseq
            lines.append(f"ASEQ {a0} {a1} {da}")
        else:
            lines.append(f"ALFA {alpha}")
        lines += [
            "PACC",               # close polar
            "",                   # leave OPER on some builds
            "QUIT",
        ]
        script = "\r\n".join(lines) + "\r\n"

        cp = subprocess.run(
            [XFOIL_EXE],
            input=script,
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=timeout,
            cwd=tmp,
            check=False,
        )

        if KEEP_TEMP:
            with open(os.path.join(tmp, "xfoil_script.inp"), "w", newline="") as f: f.write(script)
            with open(os.path.join(tmp, "stdout.txt"), "w", newline="") as f: f.write(cp.stdout)
            with open(os.path.join(tmp, "stderr.txt"), "w", newline="") as f: f.write(cp.stderr)
            print("[DEBUG] Temp dir:", tmp)
            print("[DEBUG] XFOIL stdout tail:\n", "\n".join(cp.stdout.splitlines()[-12:]))

        # 1) Try polar
        rows = parse_polar_file(polar_path)
        if rows:
            return rows

        # 2) Fallback to stdout
        sp = parse_stdout_for_singlepoint(cp.stdout)
        return sp

    finally:
        if not KEEP_TEMP:
            shutil.rmtree(tmp, ignore_errors=True)

# ---------------------- Main: single-airfoil test ----------------------
if __name__ == "__main__":
    # Build NACA 2412 and write .dat
    xu,yu,xl,yl = naca_4digit_coords(m=0.02, p=0.4, t=0.12, n=200)
    dat_path = os.path.abspath("naca2412_test.dat")
    write_xfoil_dat(dat_path, xu,yu,xl,yl)
    print("Wrote:", dat_path)

    # Run XFOIL once
    results = run_xfoil_once(dat_path)

    if not results:
        print("\n❌ No data parsed (neither polar nor stdout). Open the temp folder above and check:")
        print("   - xfoil_script.inp (commands sent)")
        print("   - stdout.txt (look for 'a = ... CL = ...' or polar prompts)")
    else:
        print("\n✅ Parsed data:")
        for (a,CL,CD,CM) in results:
            print(f"  alpha={a:6.2f} deg | CL={CL:7.4f} | CD={CD:9.6f} | CM={CM:7.4f}")
# ----------------------------------------------------------------------


Wrote: C:\Users\User\naca2412_test.dat
[DEBUG] Temp dir: C:\Users\User\AppData\Local\Temp\xf_test_l24hten7
[DEBUG] XFOIL stdout tail:
  Polar save file will NOT be written

Enter  polar dump filename  OR  <return> for no file   s>  
 New polar dump file available

 Polar accumulation enabled

.OPERva   c>  
 Polar accumulation disabled

.OPERv   c>  
 XFOIL   c>  

❌ No data parsed (neither polar nor stdout). Open the temp folder above and check:
   - xfoil_script.inp (commands sent)
   - stdout.txt (look for 'a = ... CL = ...' or polar prompts)


In [21]:
# ---------------------- simple_xfoil_stdout_only.py ----------------------
import os, re, shutil, tempfile, subprocess
import numpy as np

# ======================= CONFIG =========================================
XFOIL_EXE = r"Downloads/XFOIL6.99/xfoil.exe"   # <-- your path here
RE        = 500_000
NCRIT     = 9
MACH      = 0.0
ITER_CAP  = 200
TIMEOUT   = 60
KEEP_TEMP = True   # keep temp dir so you can open stdout.txt/script
# =======================================================================

# ---------------------- NACA 4-digit -----------------------------------
def naca_4digit_coords(m=0.02, p=0.4, t=0.12, n=200):
    beta = np.linspace(0.0, np.pi, n)
    x = 0.5*(1 - np.cos(beta))  # cosine spacing
    yt = 5.0*t*(0.2969*np.sqrt(np.clip(x,1e-12,None)) - 0.1260*x
                -0.3516*x**2 + 0.2843*x**3 - 0.1015*x**4)
    yc = np.zeros_like(x); dyc = np.zeros_like(x)
    if m > 0 and 0 < p < 1:
        m1 = x < p
        m2 = ~m1
        yc[m1]  = m/p**2*(2*p*x[m1] - x[m1]**2)
        dyc[m1] = 2*m/p**2*(p - x[m1])
        yc[m2]  = m/(1-p)**2*((1-2*p) + 2*p*x[m2] - x[m2]**2)
        dyc[m2] = 2*m/(1-p)**2*(p - x[m2])
    th = np.arctan(dyc)
    xu = x - yt*np.sin(th); yu = yc + yt*np.cos(th)
    xl = x + yt*np.sin(th); yl = yc - yt*np.cos(th)
    return xu, yu, xl, yl

def write_xfoil_dat(path, xu,yu,xl,yl):
    with open(path, "w") as f:
        f.write("NACA4\n")
        # Upper TE->LE
        for i in range(len(xu)-1, -1, -1):
            f.write(f"{xu[i]:.6f} {yu[i]:.6f}\n")
        # Lower LE->TE
        for i in range(len(xl)):
            f.write(f"{xl[i]:.6f} {yl[i]:.6f}\n")

# ---------------------- stdout parsers (no polar) -----------------------
# Expect lines like:
#   a =  4.000  CL = 0.6344
#   Cm = -0.0229  CD = 0.00470
_RE_A_CL = re.compile(r"a\s*=\s*([+-]?\d+(?:\.\d*)?)\s+.*?CL\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CD   = re.compile(r"\bCD\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CM   = re.compile(r"\bC[Mm]\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)

def parse_stdout_all(stdout_text):
    """Return list of (alpha, CL, CD, CM) by stitching ALFA prints from stdout."""
    rows = []
    cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    for line in stdout_text.splitlines():
        m1 = _RE_A_CL.search(line)
        if m1:
            # flush previous if complete
            if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
                rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
                cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
            cur["a"]  = float(m1.group(1))
            cur["CL"] = float(m1.group(2))
            continue
        m_cd = _RE_CD.search(line)
        if m_cd:
            try: cur["CD"] = float(m_cd.group(1))
            except: pass
        m_cm = _RE_CM.search(line)
        if m_cm:
            try: cur["CM"] = float(m_cm.group(1))
            except: pass
        if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
            rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
            cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
        rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
    return rows

# ---------------------- XFOIL runner (stdout-only) ----------------------
def run_xfoil_stdout(dat_file, alphas=(4.0,), re_num=RE, ncrit=NCRIT,
                     mach=MACH, iter_cap=ITER_CAP, timeout=TIMEOUT):
    """
    Run XFOIL with ONLY stdout parsing (no PACC/polar).
    - Works in a temp dir
    - CRLF line endings
    - Exits VPAR submenu via blank line
    - Sends one ALFA per AoA requested
    Returns list[(alpha, CL, CD, CM)] in order they were observed.
    """
    tmp = tempfile.mkdtemp(prefix="xf_stdout_")
    try:
        local_dat = os.path.join(tmp, "foil.dat")
        shutil.copyfile(dat_file, local_dat)

        lines = [
            "LOAD foil.dat",
            "PANE",
            "OPER",
            f"VISC {int(re_num)}",
            "VPAR",
            f"N {int(ncrit)}",
            "",                    # exit VPAR submenu
            f"MACH {mach}",
            f"ITER {int(iter_cap)}",
        ]
        for a in alphas:
            lines.append(f"ALFA {float(a)}")
        lines += [
            "",                    # leave OPER on some builds
            "QUIT",
        ]
        script = "\r\n".join(lines) + "\r\n"

        cp = subprocess.run(
            [XFOIL_EXE],
            input=script,
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=timeout,
            cwd=tmp,
            check=False,
        )

        if KEEP_TEMP:
            with open(os.path.join(tmp, "xfoil_script.inp"), "w", newline="") as f: f.write(script)
            with open(os.path.join(tmp, "stdout.txt"), "w", newline="") as f: f.write(cp.stdout)
            with open(os.path.join(tmp, "stderr.txt"), "w", newline="") as f: f.write(cp.stderr)
            print("[DEBUG] Temp dir:", tmp)
            tail = "\n".join(cp.stdout.splitlines()[-16:])
            print("[DEBUG] XFOIL stdout tail:\n" + tail)

        return parse_stdout_all(cp.stdout)

    finally:
        if not KEEP_TEMP:
            shutil.rmtree(tmp, ignore_errors=True)

# ---------------------- Main: test once ---------------------------------
if __name__ == "__main__":
    # Build NACA 2412 and write .dat
    xu,yu,xl,yl = naca_4digit_coords(m=0.02, p=0.4, t=0.12, n=200)
    dat_path = os.path.abspath("naca2412_test.dat")
    write_xfoil_dat(dat_path, xu,yu,xl,yl)
    print("Wrote:", dat_path)

    # Try single alpha or a small list
    # results = run_xfoil_stdout(dat_path, alphas=(4.0,))               # single
    results = run_xfoil_stdout(dat_path, alphas=(-2, 0, 2, 4, 6, 8))     # short sweep

    if not results:
        print("\n❌ No CL/CD parsed from stdout. Open the temp folder and inspect stdout.txt.")
    else:
        print("\n✅ Parsed from stdout:")
        for (a,CL,CD,CM) in results:
            print(f"  a={a:6.2f}°  CL={CL:8.5f}  CD={CD:10.7f}  CM={CM:8.5f}")
# ------------------------------------------------------------------------


Wrote: C:\Users\User\naca2412_test.dat
[DEBUG] Temp dir: C:\Users\User\AppData\Local\Temp\xf_stdout_dfdue4ue
[DEBUG] XFOIL stdout tail:
 Side 1  free  transition at x/c =  0.0823   35
 Side 2 forced transition at x/c =  0.9999   69

   9   rms: 0.5146E-03   max: 0.9706E-02   C at   35  1
       a =  8.000      CL =  1.0697
      Cm = -0.0401     CD =  0.01463   =>   CDf =  0.00696    CDp =  0.00767

 Side 1  free  transition at x/c =  0.0822   35
 Side 2 forced transition at x/c =  0.9999   69

  10   rms: 0.2678E-05   max: -.2025E-04   C at   35  1
       a =  8.000      CL =  1.0697
      Cm = -0.0401     CD =  0.01463   =>   CDf =  0.00697    CDp =  0.00767

.OPERv   c>  
 XFOIL   c>  

✅ Parsed from stdout:
  a= -2.00°  CL=-0.00460  CD= 0.0077800  CM=-0.04720
  a= -2.00°  CL=-0.01130  CD= 0.0078700  CM=-0.04520
  a= -2.00°  CL= 0.00150  CD= 0.0078200  CM=-0.04830
  a= -2.00°  CL= 0.00990  CD= 0.0074200  CM=-0.05110
  a= -2.00°  CL= 0.01500  CD= 0.0075100  CM=-0.05200
  a= -2.00°  C

In [2]:
# ------------------- stdout_only_optimizer.py -------------------
import os, re, shutil, tempfile, subprocess
from functools import lru_cache
from datetime import datetime

import numpy as np
from scipy.optimize import differential_evolution

# ================== CONFIG ==================
XFOIL_EXE = r"Downloads/XFOIL6.99/xfoil.exe"   # <--- set your path
RE        = 500_000
NCRIT     = 9
MACH      = 0.0
ITER_CAP  = 500
TIMEOUT   = 60
KEEP_TEMP = False

ALPHAS = np.arange(-2.0, 9.0, 1.0)  # -2..8 deg

# DE settings
BOUNDS = [(0.00, 0.09), (0.30, 0.70), (0.08, 0.18)]  # m, p, t
GENS   = 120
POPSZ  = 15
F_MUT  = 0.7
CR     = 0.9
SEED   = 42

BAD_FIT = 1e3  # finite penalty to keep DE stable
# ============================================

# --------------- NACA 4-digit ----------------
def naca_4digit_coords(m=0.02, p=0.4, t=0.12, n=200):
    beta = np.linspace(0.0, np.pi, n)
    x = 0.5*(1 - np.cos(beta))  # cosine spacing
    yt = 5.0*t*(0.2969*np.sqrt(np.clip(x,1e-12,None)) - 0.1260*x
                -0.3516*x**2 + 0.2843*x**3 - 0.1015*x**4)
    yc = np.zeros_like(x); dyc = np.zeros_like(x)
    if m > 0 and 0 < p < 1:
        m1 = x < p
        m2 = ~m1
        yc[m1]  = m/p**2*(2*p*x[m1] - x[m1]**2)
        dyc[m1] = 2*m/p**2*(p - x[m1])
        yc[m2]  = m/(1-p)**2*((1-2*p) + 2*p*x[m2] - x[m2]**2)
        dyc[m2] = 2*m/(1-p)**2*(p - x[m2])
    th = np.arctan(dyc)
    xu = x - yt*np.sin(th); yu = yc + yt*np.cos(th)
    xl = x + yt*np.sin(th); yl = yc - yt*np.cos(th)
    return xu, yu, xl, yl

def write_xfoil_dat(path, xu,yu,xl,yl):
    with open(path, "w") as f:
        f.write("NACA4\n")
        for i in range(len(xu)-1, -1, -1):  # Upper TE->LE
            f.write(f"{xu[i]:.6f} {yu[i]:.6f}\n")
        for i in range(len(xl)):            # Lower LE->TE
            f.write(f"{xl[i]:.6f} {yl[i]:.6f}\n")

# --------------- stdout parser ----------------
# Matches lines like:
#   a =  4.000  CL = 0.6344
#   Cm = -0.0229  CD = 0.00470
_RE_A_CL = re.compile(r"a\s*=\s*([+-]?\d+(?:\.\d*)?)\s+.*?CL\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CD   = re.compile(r"\bCD\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CM   = re.compile(r"\bC[Mm]\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)

def parse_stdout_all(stdout_text):
    """Collect (alpha, CL, CD, CM) records from stdout, stitching the two-line format."""
    rows = []
    cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    for line in stdout_text.splitlines():
        m1 = _RE_A_CL.search(line)
        if m1:
            if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
                rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
            cur = {"a": float(m1.group(1)), "CL": float(m1.group(2)), "CD": None, "CM": 0.0}
            continue
        m_cd = _RE_CD.search(line)
        if m_cd:
            try: cur["CD"] = float(m_cd.group(1))
            except: pass
        m_cm = _RE_CM.search(line)
        if m_cm:
            try: cur["CM"] = float(m_cm.group(1))
            except: pass
        if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
            rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
            cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
        rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
    return rows

# --------------- XFOIL runner (stdout only) ---------------
def run_xfoil_stdout(dat_file, alphas, re_num=RE, ncrit=NCRIT, mach=MACH,
                     iter_cap=ITER_CAP, timeout=TIMEOUT, keep_temp=KEEP_TEMP):
    """
    Run XFOIL with only stdout parsing (no PACC/polar). Returns list[(a, CL, CD, CM)].
    """
    tmp = tempfile.mkdtemp(prefix="xf_stdout_")
    try:
        local_dat = os.path.join(tmp, "foil.dat")
        shutil.copyfile(dat_file, local_dat)
        lines = [
            "LOAD foil.dat",
            "PANE",
            "OPER",
            f"VISC {int(re_num)}",
            "VPAR",
            f"N {int(ncrit)}",
            "",                    # exit VPAR submenu
            f"MACH {mach}",
            f"ITER {int(iter_cap)}",
        ]
        for a in alphas:
            lines.append(f"ALFA {float(a)}")
        lines += ["", "QUIT"]     # leave OPER (some builds), then quit
        script = "\r\n".join(lines) + "\r\n"

        cp = subprocess.run(
            [XFOIL_EXE],
            input=script,
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=timeout,
            cwd=tmp,
            check=False,
        )

        if keep_temp:
            with open(os.path.join(tmp, "xfoil_script.inp"), "w", newline="") as f: f.write(script)
            with open(os.path.join(tmp, "stdout.txt"), "w", newline="") as f: f.write(cp.stdout)

        return parse_stdout_all(cp.stdout)

    finally:
        if not keep_temp:
            shutil.rmtree(tmp, ignore_errors=True)

# --------------- Objective & optimization ---------------
eval_count = 0
best_ld_seen = -np.inf

@lru_cache(maxsize=4096)
def evaluate_mpt(m, p, t):
    """Return best (L/D, a*, CL, CD) over ALPHAS for this (m,p,t), or None on failure."""
    # Build airfoil file once
    tmp_dir = tempfile.mkdtemp(prefix="foil_")
    try:
        dat_path = os.path.join(tmp_dir, "airfoil.dat")
        xu,yu,xl,yl = naca_4digit_coords(m, p, t, n=200)
        write_xfoil_dat(dat_path, xu,yu,xl,yl)

        rows = run_xfoil_stdout(dat_path, ALPHAS)
        if not rows:
            return None

        # XFOIL often prints several lines per alpha.
        # Strategy: per-alpha, take the *last* line (usually converged), then pick max L/D.
        per_alpha_last = {}
        for (a, CL, CD, CM) in rows:
            if CD is not None and CD > 0:
                per_alpha_last[a] = (CL, CD, CM)  # overwrites earlier at same a

        if not per_alpha_last:
            return None

        best = None
        for a, (CL, CD, CM) in per_alpha_last.items():
            ld = CL / CD if CD > 0 else -np.inf
            if best is None or ld > best[0]:
                best = (ld, a, CL, CD)

        return best
    finally:
        shutil.rmtree(tmp_dir, ignore_errors=True)

def objective(x):
    global eval_count, best_ld_seen
    eval_count += 1
    m, p, t = float(x[0]), float(x[1]), float(x[2])

    # Feasibility guard (helps XFOIL)
    if not (0.00 <= m <= 0.09 and 0.30 <= p <= 0.70 and 0.08 <= t <= 0.18):
        print(f"Eval {eval_count:4d}: bounds reject m={m:.3f}, p={p:.3f}, t={t:.3f}")
        return BAD_FIT

    res = evaluate_mpt(round(m,4), round(p,4), round(t,4))
    if res is None:
        print(f"Eval {eval_count:4d}: XFOIL failed m={m:.3f}, p={p:.3f}, t={t:.3f}")
        return BAD_FIT

    ld, a_star, CL, CD = res
    best_ld_seen = max(best_ld_seen, ld)
    tag = f"{int(m*100):02d}{int(p*10):1d}{int(t*100):02d}"
    print(f"Eval {eval_count:4d}: NACA {tag} | α*={a_star:4.1f} | CL={CL:6.3f} | CD={CD:8.5f} | L/D={ld:7.2f} | Best={best_ld_seen:7.2f}")
    return -ld  # SciPy minimizes

# --------------- Main ----------------
def main():
    print("="*70)
    print("AIRFOIL OPTIMIZATION WITH XFOIL (stdout-only, AoA sweep)")
    print("="*70)
    print(f"Re={RE}, Ncrit={NCRIT}, Mach={MACH}")
    print(f"AoA list: {list(ALPHAS)}")
    print(f"DE: gens={GENS}, popsize={POPSZ}, seed={SEED}")
    print(f"Bounds m{BOUNDS[0]} p{BOUNDS[1]} t{BOUNDS[2]}")
    print("="*70, "\n")

    start = datetime.now()
    result = differential_evolution(
        objective,
        bounds=BOUNDS,
        maxiter=GENS,
        popsize=POPSZ,
        strategy="best1bin",
        mutation=F_MUT,
        recombination=CR,
        seed=SEED,
        workers=1,
        updating="deferred",
        disp=False,
    )
    dur = (datetime.now() - start).total_seconds()

    best_m, best_p, best_t = result.x
    best = evaluate_mpt(round(best_m,4), round(best_p,4), round(best_t,4))
    if best is None:
        print("\n(No successful evaluations to report.)")
        return

    ld, a_star, CL, CD = best
    naca = f"{int(best_m*100):02d}{int(best_p*10):1d}{int(best_t*100):02d}"

    print("\n" + "="*70)
    print("OPTIMIZATION COMPLETE")
    print("="*70)
    print(f"Time: {dur:.1f}s  Evals: {eval_count}")
    print(f"Best NACA: {naca}")
    print(f"  m={best_m:.4f} ({best_m*100:.2f}%)")
    print(f"  p={best_p:.4f} ({best_p*100:.1f}% chord)")
    print(f"  t={best_t:.4f} ({best_t*100:.2f}%)")
    print(f"Best L/D={ld:.2f} at α*={a_star:.1f}°  (CL={CL:.3f}, CD={CD:.5f})")
    print("="*70)

if __name__ == "__main__":
    main()
# -----------------------------------------------------------


AIRFOIL OPTIMIZATION WITH XFOIL (stdout-only, AoA sweep)
Re=500000, Ncrit=9, Mach=0.0
AoA list: [np.float64(-2.0), np.float64(-1.0), np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0)]
DE: gens=120, popsize=15, seed=42
Bounds m(0.0, 0.09) p(0.3, 0.7) t(0.08, 0.18)

Eval    1: NACA 03614 | α*= 3.0 | CL= 0.860 | CD= 0.00849 | L/D= 101.26 | Best= 101.26
Eval    2: NACA 02416 | α*= 5.0 | CL= 0.894 | CD= 0.01015 | L/D=  88.07 | Best= 101.26
Eval    3: NACA 03415 | α*= 6.0 | CL= 1.099 | CD= 0.01089 | L/D= 100.88 | Best= 101.26


TimeoutExpired: Command '['Downloads/XFOIL6.99/xfoil.exe']' timed out after 60 seconds

In [ ]:
# =========================== stdout_only_optimizer.py ===========================
import os, re, shutil, tempfile, subprocess
from functools import lru_cache
from datetime import datetime

import numpy as np
from scipy.optimize import differential_evolution

# ==============================================================================
# CONFIG
# ==============================================================================
# ⬇️ SET THIS to your XFOIL path (Windows example shown)
XFOIL_EXE = r"Downloads/XFOIL6.99/xfoil.exe"

# Aerodynamic settings
RE        = 500_000
NCRIT     = 9
MACH      = 0.0
ITER_CAP  = 1000            # a bit higher helps convergence
TIMEOUT   = 20             # seconds per XFOIL call (hard wall)
KEEP_TEMP = False          # True = keep temp folder for debugging

# Angle-of-attack list (stdout-only runner calls ALFA for each value)
# You can start smaller while debugging: ALPHAS = [2.0, 4.0, 6.0]
ALPHAS = list(np.arange(-2.0, 9.0, 1.0))   # -2, -1, ..., 8 deg

# Differential Evolution (DE) settings
# Note: total individuals per generation = popsize * dims (dims=3 here)
BOUNDS   = [(0.00, 0.09), (0.30, 0.70), (0.08, 0.18)]  # (m, p, t)
GENS     = 60     # use 25–60 while debugging; increase later
POPSZ    = 9      # popsize multiplier (individuals = POPSZ * 3)
F_MUT    = 0.7
CR       = 0.9
SEED     = 42

# Penalty for failed evaluations (finite so DE remains stable)
BAD_FIT  = 1e3

# ==============================================================================
# NACA 4-digit geometry
# ==============================================================================
def naca_4digit_coords(m=0.02, p=0.4, t=0.12, n=200):
    """Return (xu, yu, xl, yl) using cosine spacing."""
    beta = np.linspace(0.0, np.pi, n)
    x = 0.5 * (1 - np.cos(beta))  # cosine spacing in [0,1]

    yt = 5.0 * t * (
        0.2969 * np.sqrt(np.clip(x, 1e-12, None))
        - 0.1260 * x
        - 0.3516 * x**2
        + 0.2843 * x**3
        - 0.1015 * x**4
    )

    yc = np.zeros_like(x)
    dyc = np.zeros_like(x)
    if m > 0 and 0 < p < 1:
        mask1 = x < p
        mask2 = ~mask1
        # 0 <= x < p
        yc[mask1]  = m / p**2 * (2 * p * x[mask1] - x[mask1]**2)
        dyc[mask1] = 2 * m / p**2 * (p - x[mask1])
        # p <= x <= 1
        yc[mask2]  = m / (1 - p)**2 * ((1 - 2*p) + 2*p*x[mask2] - x[mask2]**2)
        dyc[mask2] = 2 * m / (1 - p)**2 * (p - x[mask2])

    theta = np.arctan(dyc)

    xu = x - yt * np.sin(theta)
    yu = yc + yt * np.cos(theta)
    xl = x + yt * np.sin(theta)
    yl = yc - yt * np.cos(theta)
    return xu, yu, xl, yl


def write_xfoil_dat(path, xu, yu, xl, yl):
    """
    XFOIL-friendly .dat:
      - Upper surface: TE -> LE
      - Lower surface: LE -> TE
    """
    with open(path, "w") as f:
        f.write("NACA4\n")
        # Upper TE -> LE
        for i in range(len(xu) - 1, -1, -1):
            f.write(f"{xu[i]:.6f} {yu[i]:.6f}\n")
        # Lower LE -> TE
        for i in range(len(xl)):
            f.write(f"{xl[i]:.6f} {yl[i]:.6f}\n")

# ==============================================================================
# Stdout parser (no polar files)
# ==============================================================================
# We expect pairs of lines like:
#   a =  4.000  CL = 0.6344
#   Cm = -0.0229  CD = 0.00470
_RE_A_CL = re.compile(
    r"a\s*=\s*([+-]?\d+(?:\.\d*)?)\s+.*?CL\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)",
    re.I,
)
_RE_CD = re.compile(r"\bCD\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CM = re.compile(r"\bC[Mm]\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)

def parse_stdout_all(stdout_text):
    """
    Stitch 'a & CL' with 'Cm & CD' (possibly on the next line).
    Return list of (alpha, CL, CD, CM).
    """
    rows = []
    cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    for line in stdout_text.splitlines():
        m1 = _RE_A_CL.search(line)
        if m1:
            # flush previous if complete==================================
# We expect pairs of lines like:
            if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
                rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
            cur = {"a": float(m1.group(1)), "CL": float(m1.group(2)), "CD": None, "CM": 0.0}
            continue

        m_cd = _RE_CD.search(line)
        if m_cd:
            try:
                cur["CD"] = float(m_cd.group(1))
            except:
                pass

        m_cm = _RE_CM.search(line)
        if m_cm:
            try:
                cur["CM"] = float(m_cm.group(1))
            except:
                pass

        if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
            rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))
            cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}

    if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
        rows.append((cur["a"], cur["CL"], cur["CD"], cur["CM"]))

    return rows

# ==============================================================================
# XFOIL runner (stdout-only, robust to stalls)
# ==============================================================================
def run_xfoil_stdout(dat_file, alphas, re_num=RE, ncrit=NCRIT, mach=MACH,
                     iter_cap=ITER_CAP, timeout=TIMEOUT, keep_temp=KEEP_TEMP):
    """
    Run XFOIL once with ONLY stdout parsing (no PACC/polar).
    Returns list[(alpha, CL, CD, CM)] in the order observed.
    """
    tmp = tempfile.mkdtemp(prefix="xf_stdout_")
    try:
        local_dat = os.path.join(tmp, "foil.dat")
        shutil.copyfile(dat_file, local_dat)

        lines = [
            "LOAD foil.dat",
            "PANE",
            "OPER",
            f"VISC {int(re_num)}",
            "VPAR",
            f"N {int(ncrit)}",
            "",                    # exit VPAR submenu back to OPER
            f"MACH {mach}",
            f"ITER {int(iter_cap)}",
            f"ALFA {4.0}",
            "QUIT",
        ]
        script = "\r\n".join(lines) + "\r\n"

        try:
            cp = subprocess.run(
                [XFOIL_EXE],
                input=script,
                text=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=timeout,
                cwd=tmp,
                check=False,
            )
        except subprocess.TimeoutExpired:
            return []  # signal failure to caller
        except FileNotFoundError:
            print("❌ XFOIL executable not found. Check XFOIL_EXE path.")
            return []

        if keep_temp:
            with open(os.path.join(tmp, "xfoil_script.inp"), "w", newline="") as f: f.write(script)
            with open(os.path.join(tmp, "stdout.txt"), "w", newline="") as f: f.write(cp.stdout)
            with open(os.path.join(tmp, "stderr.txt"), "w", newline="") as f: f.write(cp.stderr)

        return parse_stdout_all(cp.stdout)

    finally:
        if not keep_temp:
            shutil.rmtree(tmp, ignore_errors=True)

# ==============================================================================
# Objective and optimization
# ==============================================================================
eval_count = 0
best_ld_seen = -np.inf

@lru_cache(maxsize=4096)
def evaluate_mpt(m, p, t):
    """
    Build airfoil (m,p,t), run XFOIL across ALPHAS, and return
    (best_L_over_D, a_star, CL, CD). Returns None on failure.
    """
    tmp_dir = tempfile.mkdtemp(prefix="foil_")
    try:
        dat_path = os.path.join(tmp_dir, "airfoil.dat")
        xu, yu, xl, yl = naca_4digit_coords(m, p, t, n=200)
        write_xfoil_dat(dat_path, xu, yu, xl, yl)

        rows = run_xfoil_stdout(dat_path, ALPHAS)
        if not rows:
            return None

        # XFOIL may print multiple lines per same alpha; keep the last (usually converged).
        per_alpha_last = {}
        for (a, CL, CD, CM) in rows:
            if CD is not None and CD > 0:
                per_alpha_last[a] = (CL, CD, CM)

        if not per_alpha_last:
            return None

        best = None
        for a, (CL, CD, CM) in per_alpha_last.items():
            ld = CL / CD if CD > 0 else -np.inf
            if best is None or ld > best[0]:
                best = (ld, a, CL, CD)

        return best
    finally:
        shutil.rmtree(tmp_dir, ignore_errors=True)


def objective(x):
    m, p, t = x
    x_u, y_u, x_l, y_l = generate_naca_4digit(m, p, t)
    dat = "airfoil_temp.dat"
    write_airfoil_file(dat, x_u, y_u, x_l, y_l)

    res = run_xfoil_at_alpha(dat, Re=500000, alpha=4.0)
    if res is None:
        return BAD_FIT
    
    a, CL, CD, CM = res
    if CD <= 0:
        return BAD_FIT
    
    return -(CL / CD)

# ==============================================================================
# Main
# ==============================================================================
def main():
    print("="*70)
    print("AIRFOIL OPTIMIZATION WITH XFOIL (stdout-only, AoA sweep)")
    print("="*70)
    print(f"Re={RE}, Ncrit={NCRIT}, Mach={MACH}")
    print(f"AoA list: {ALPHAS}")
    print(f"DE: gens={GENS}, popsize={POPSZ}, seed={SEED}")
    print(f"Bounds m{BOUNDS[0]} p{BOUNDS[1]} t{BOUNDS[2]}")
    print("="*70, "\n")

    start = datetime.now()
    result = differential_evolution(
        objective,
        bounds=BOUNDS,
        maxiter=GENS,
        popsize=POPSZ,
        strategy="best1bin",
        mutation=F_MUT,
        recombination=CR,
        seed=SEED,
        workers=1,               # <- one worker; easier to debug and print
        updating="immediate",    # <- print progress in order, one eval at a time
        disp=False,
    )
    dur = (datetime.now() - start).total_seconds()

    # Evaluate the reported best (sometimes history holds an even better one, but stdout-only is fine)
    best_m, best_p, best_t = result.x
    best = evaluate_mpt(round(best_m, 4), round(best_p, 4), round(best_t, 4))
    if best is None:
        print("\n(No successful evaluations to report.)")
        return

    ld, a_star, CL, CD = best
    naca = f"{int(best_m*100):02d}{int(best_p*10):1d}{int(best_t*100):02d}"

    print("\n" + "="*70)
    print("OPTIMIZATION COMPLETE")
    print("="*70)
    print(f"Time: {dur:.1f}s  Evals: {eval_count}")
    print(f"Best NACA: {naca}")
    print(f"  m={best_m:.4f} ({best_m*100:.2f}%)")
    print(f"  p={best_p:.4f} ({best_p*100:.1f}% chord)")
    print(f"  t={best_t:.4f} ({best_t*100:.2f}%)")
    print(f"Best L/D={ld:.2f} at α*={a_star:.1f}°  (CL={CL:.3f}, CD={CD:.5f})")
    print("="*70)

if __name__ == "__main__":
    main()
# ==============================================================================


In [1]:
# ======================= de_naca_stdout_single_aoa.py =======================
import os, re, shutil, tempfile, subprocess
from functools import lru_cache
from datetime import datetime

import numpy as np
from scipy.optimize import differential_evolution

# ---------------------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------------------
XFOIL_EXE = r"Downloads/XFOIL6.99/xfoil.exe"  # <-- set your path

# Aerodynamics / XFOIL
RE        = 500_000
NCRIT     = 9
MACH      = 0.0
ITER_CAP  = 300           # 200–400 is typical; higher isn't always better
TIMEOUT   = 20            # seconds per XFOIL call wall-time
KEEP_TEMP = False         # True to keep temp folder for debugging

# Single design AoA (constant)
ALPHA0    = 4.0           # degrees — change this to any constant you want

# Differential Evolution (DE)
BOUNDS = [(0.00, 0.09), (0.30, 0.70), (0.08, 0.18)]   # m, p, t
GENS   = 60       # use 20–60 while testing; increase later
POPSZ  = 9        # individuals per gen = POPSZ * dims (dims=3) = 27
F_MUT  = 0.7
CR     = 0.9
SEED   = 42
BAD_FIT = 1e3     # finite penalty so DE keeps running

# ---------------------------------------------------------------------------
# NACA 4-digit geometry (cosine spacing)
# ---------------------------------------------------------------------------
def naca_4digit_coords(m, p, t, n=200):
    beta = np.linspace(0.0, np.pi, n)
    x = 0.5 * (1 - np.cos(beta))  # cosine spacing

    yt = 5.0 * t * (
        0.2969 * np.sqrt(np.clip(x, 1e-12, None))
        - 0.1260 * x
        - 0.3516 * x**2
        + 0.2843 * x**3
        - 0.1015 * x**4
    )

    yc  = np.zeros_like(x)
    dyc = np.zeros_like(x)
    if m > 0 and 0 < p < 1:
        mask1 = x < p
        mask2 = ~mask1
        yc[mask1]  = m / p**2 * (2*p*x[mask1] - x[mask1]**2)
        dyc[mask1] = 2*m / p**2 * (p - x[mask1])
        yc[mask2]  = m / (1 - p)**2 * ((1 - 2*p) + 2*p*x[mask2] - x[mask2]**2)
        dyc[mask2] = 2*m / (1 - p)**2 * (p - x[mask2])

    theta = np.arctan(dyc)

    xu = x - yt * np.sin(theta)
    yu = yc + yt * np.cos(theta)
    xl = x + yt * np.sin(theta)
    yl = yc - yt * np.cos(theta)
    return xu, yu, xl, yl

def write_xfoil_dat(path, xu, yu, xl, yl):
    """XFOIL-friendly ordering: upper TE->LE then lower LE->TE."""
    with open(path, "w") as f:
        f.write("NACA4\n")
        for i in range(len(xu) - 1, -1, -1):   # upper TE -> LE
            f.write(f"{xu[i]:.6f} {yu[i]:.6f}\n")
        for i in range(len(xl)):               # lower LE -> TE
            f.write(f"{xl[i]:.6f} {yl[i]:.6f}\n")

# ---------------------------------------------------------------------------
# Stdout parser (single-point; stitches 2-line format)
# ---------------------------------------------------------------------------
_RE_A_CL = re.compile(
    r"a\s*=\s*([+-]?\d+(?:\.\d*)?)\s+.*?CL\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)",
    re.I,
)
_RE_CD = re.compile(r"\bCD\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)
_RE_CM = re.compile(r"\bC[Mm]\s*=\s*([+-]?\d+(?:\.\d*)?(?:[Ee][+-]?\d+)?)", re.I)

def parse_stdout_single(stdout_text):
    """
    Return (alpha, CL, CD, CM) for the last assembled record in stdout.
    XFOIL often prints two lines per evaluation:
      line 1: a & CL
      line 2: CM & CD (order may vary)
    """
    cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    last = None
    for line in stdout_text.splitlines():
        m1 = _RE_A_CL.search(line)
        if m1:
            if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
                last = (cur["a"], cur["CL"], cur["CD"], cur["CM"])
            cur = {"a": float(m1.group(1)), "CL": float(m1.group(2)), "CD": None, "CM": 0.0}
            continue
        m_cd = _RE_CD.search(line)
        if m_cd:
            try: cur["CD"] = float(m_cd.group(1))
            except: pass
        m_cm = _RE_CM.search(line)
        if m_cm:
            try: cur["CM"] = float(m_cm.group(1))
            except: pass
        if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
            last = (cur["a"], cur["CL"], cur["CD"], cur["CM"])
            cur = {"a": None, "CL": None, "CD": None, "CM": 0.0}
    if cur["a"] is not None and cur["CL"] is not None and cur["CD"] is not None:
        last = (cur["a"], cur["CL"], cur["CD"], cur["CM"])
    return last

# ---------------------------------------------------------------------------
# XFOIL runner (single ALFA, stdout only)
# ---------------------------------------------------------------------------
def run_xfoil_at_alpha(dat_file, alpha=ALPHA0, re_num=RE, ncrit=NCRIT, mach=MACH,
                       iter_cap=ITER_CAP, timeout=TIMEOUT, keep_temp=KEEP_TEMP):
    """
    Run a single operating point (constant AoA) and parse stdout.
    Returns (alpha, CL, CD, CM) or None on failure.
    """
    tmp = tempfile.mkdtemp(prefix="xf_sp_")
    try:
        local_dat = os.path.join(tmp, "foil.dat")
        shutil.copyfile(dat_file, local_dat)

        lines = [
            "LOAD foil.dat",
            "PANE",
            "OPER",
            f"VISC {int(re_num)}",
            "VPAR",
            f"N {int(ncrit)}",
            "",                         # exit VPAR submenu
            f"MACH {mach}",
            f"ITER {int(iter_cap)}",
            f"ALFA {float(alpha)}",
            "",                         # return to main (some builds)
            "QUIT",
        ]
        script = "\r\n".join(lines) + "\r\n"

        try:
            cp = subprocess.run(
                [XFOIL_EXE],
                input=script,
                text=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=timeout,
                cwd=tmp,
                check=False,
            )
        except subprocess.TimeoutExpired:
            return None
        except FileNotFoundError:
            print("❌ XFOIL executable not found. Check XFOIL_EXE.")
            return None
        except Exception:
            return None

        if keep_temp:
            open(os.path.join(tmp, "xfoil_script.inp"), "w", newline="").write(script)
            open(os.path.join(tmp, "stdout.txt"), "w", newline="").write(cp.stdout)
            open(os.path.join(tmp, "stderr.txt"), "w", newline="").write(cp.stderr)

        return parse_stdout_single(cp.stdout)
    finally:
        if not keep_temp:
            shutil.rmtree(tmp, ignore_errors=True)

# ---------------------------------------------------------------------------
# Objective (maximize L/D at ALPHA0) — SciPy minimizes, so return -L/D
# ---------------------------------------------------------------------------
eval_count = 0
best_ld_seen = -np.inf

@lru_cache(maxsize=4096)
def evaluate_tuple(m, p, t, alpha=ALPHA0):
    """Build foil for (m,p,t), run XFOIL at fixed AoA, return (L/D, CL, CD)."""
    tmp_dir = tempfile.mkdtemp(prefix="foil_")
    try:
        dat_path = os.path.join(tmp_dir, "airfoil.dat")
        xu, yu, xl, yl = naca_4digit_coords(m, p, t, n=200)
        write_xfoil_dat(dat_path, xu, yu, xl, yl)

        res = run_xfoil_at_alpha(dat_path, alpha=alpha)
        if res is None:
            return None
        a, CL, CD, CM = res
        if CD is None or CD <= 0:
            return None
        return (CL / CD, CL, CD)
    finally:
        shutil.rmtree(tmp_dir, ignore_errors=True)

def objective(x):
    global eval_count, best_ld_seen
    eval_count += 1
    m, p, t = float(x[0]), float(x[1]), float(x[2])

    # light feasibility guard (helps avoid pathological panelings)
    if not (0.00 <= m <= 0.09 and 0.30 <= p <= 0.70 and 0.08 <= t <= 0.18):
        print(f"Eval {eval_count:4d}: bounds reject m={m:.3f}, p={p:.3f}, t={t:.3f}")
        return BAD_FIT

    key = (round(m,4), round(p,4), round(t,4))
    res = evaluate_tuple(*key, alpha=ALPHA0)
    if res is None:
        print(f"Eval {eval_count:4d}: XFOIL FAILED m={m:.3f}, p={p:.3f}, t={t:.3f}")
        return BAD_FIT

    ld, CL, CD = res
    best_ld_seen = max(best_ld_seen, ld)
    tag = f"{int(m*100):02d}{int(p*10):1d}{int(t*100):02d}"
    print(f"Eval {eval_count:4d}: NACA {tag} | α={ALPHA0:4.1f} | CL={CL:6.3f} | CD={CD:8.5f} | L/D={ld:7.2f} | Best={best_ld_seen:7.2f}")
    return -ld

# ---------------------------------------------------------------------------
# Main
# ---------------------------------------------------------------------------
def main():
    print("="*70)
    print("AIRFOIL OPTIMIZATION WITH XFOIL (stdout-only, single AoA)")
    print("="*70)
    print(f"Re={RE}, Ncrit={NCRIT}, Mach={MACH}, AoA={ALPHA0}°")
    print(f"DE: gens={GENS}, popsize={POPSZ}, seed={SEED}")
    print(f"Bounds m{BOUNDS[0]} p{BOUNDS[1]} t{BOUNDS[2]}")
    print("="*70, "\n")

    start = datetime.now()
    result = differential_evolution(
        objective,
        bounds=BOUNDS,
        maxiter=GENS,
        popsize=POPSZ,
        strategy="best1bin",
        mutation=F_MUT,
        recombination=CR,
        seed=SEED,
        workers=1,              # deterministic prints; parallelize later
        updating="immediate",
        tol=0, atol=0,          # prevent early stop if many penalties
        polish=False,           # skip final local search
        disp=False,
    )
    dur = (datetime.now() - start).total_seconds()

    best_m, best_p, best_t = result.x
    final = evaluate_tuple(round(best_m,4), round(best_p,4), round(best_t,4), alpha=ALPHA0)
    if final is None:
        print("\n(No successful evaluations to report.)")
        return

    ld, CL, CD = final
    naca = f"{int(best_m*100):02d}{int(best_p*10):1d}{int(best_t*100):02d}"
    print("\n" + "="*70)
    print("OPTIMIZATION COMPLETE")
    print("="*70)
    print(f"Time: {dur:.1f}s  Evals: {eval_count}")
    print(f"Best NACA: {naca}")
    print(f"  m={best_m:.4f} ({best_m*100:.2f}%)")
    print(f"  p={best_p:.4f} ({best_p*100:.1f}% chord)")
    print(f"  t={best_t:.4f} ({best_t*100:.2f}%)")
    print(f"Best L/D={ld:.2f} at α={ALPHA0:.1f}°  (CL={CL:.3f}, CD={CD:.5f})")
    print("="*70)

if __name__ == "__main__":
    main()
# ======================================================================


AIRFOIL OPTIMIZATION WITH XFOIL (stdout-only, single AoA)
Re=500000, Ncrit=9, Mach=0.0, AoA=4.0°
DE: gens=60, popsize=9, seed=42
Bounds m(0.0, 0.09) p(0.3, 0.7) t(0.08, 0.18)

Eval    1: NACA 03413 | α= 4.0 | CL= 0.869 | CD= 0.00891 | L/D=  97.51 | Best=  97.51
Eval    2: NACA 08612 | α= 4.0 | CL= 1.623 | CD= 0.00648 | L/D= 250.49 | Best= 250.49
Eval    3: NACA 01314 | α= 4.0 | CL= 0.605 | CD= 0.00940 | L/D=  64.32 | Best= 250.49
Eval    4: NACA 06413 | α= 4.0 | CL= 1.162 | CD= 0.01014 | L/D= 114.63 | Best= 250.49
Eval    5: NACA 06611 | α= 4.0 | CL= 1.336 | CD= 0.00901 | L/D= 148.29 | Best= 250.49
Eval    6: NACA 04417 | α= 4.0 | CL= 0.948 | CD= 0.00965 | L/D=  98.23 | Best= 250.49
Eval    7: XFOIL FAILED m=0.017, p=0.317, t=0.095
Eval    8: XFOIL FAILED m=0.028, p=0.494, t=0.084
Eval    9: NACA 02314 | α= 4.0 | CL= 0.757 | CD= 0.00913 | L/D=  82.95 | Best= 250.49
Eval   10: NACA 03316 | α= 4.0 | CL= 0.867 | CD= 0.01041 | L/D=  83.25 | Best= 250.49
Eval   11: NACA 04509 | α= 4.0 | CL=